In [1]:
import json
import time
import pandas as pd
from google.cloud import storage 
from google.cloud import bigquery
from pathlib import Path
import os

1. Extract tables from Big Query using a JOIN SQL query - combining the Binary Answers on the 'names' with the Questions
 - it is important to NOT use the Learner information from the JiveLearner to create a 'Learner' dataframe, because it results in fewer data points. Instead use a multiple-join
2. Read in the data as a dateframe
4. Process the data by:
    - choosing the last modified
    - eliminate duplicates
5. create a new dataframe which has the Learners as the index and Questions as columns
    - encode the categorical answers numerically (-2, -1, 0, 1, 2)
    - changing the answers to overwrite based on the most definitive (highest or lowest) number
4. export the table to bigquery
    - None / NaN BQ datatype
    - df = df.where(pd.notnull(df), None)

- Q 371: "Do you work outside of the home?" | 2,012
- Q 367: "Does {kid_name} attend school outside of the home?" | 1,349
- Q 368: "Do you have reliable childcare for {kid_name}?" | 1,216
- Q 24: "Do you know the name of {kid_name}'s primary doctor?" | 350
- Q 13: "Do you know your next door neighbor's first name?" | 341
- Q 25: "Does your family have health care?" | 339
- Q 18: "Does your spouse work multiple jobs?" | 335
- Q 590: "Do you have a primary doctor?" | 325
- Q 15: "Does your spouse work full-time?" | 318
- Q 41: "Are you the first person in your family to graduate college?" | 314

In [2]:
PROJ_ROOT = Path().resolve().parent
KEYS_DIR = PROJ_ROOT / 'keys' 
keys = KEYS_DIR / 'socos-quickstep-601-403a611fa9db.json'

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str(keys)

In [4]:
client = bigquery.Client()

In [5]:
join_query="""
#standardSQL
SELECT a.JiveLearner.id as learner, \
b.message.jive_id as question, \
a.answer, a.last_modified, \
a.JiveEnvelope.name, \
b.message.learner_name, \
a.tags.tag, \
a.tags.metatag, \
c.who, \
c.text, \
c.answer_type, ARRAY_TO_STRING(a.tags.tag, ', ') as answer_tag, ARRAY_TO_STRING(a.tags.metatag, ', ') as metatags
FROM `socos-quickstep-601.museathome.BinaryQuestionAnswer` as a
JOIN `socos-quickstep-601.museathome.JiveEnvelope` as b ON a.JiveEnvelope.name=b.__key__.name
JOIN `socos-quickstep-601.museathome.questions` as c ON CAST(b.message.jive_id AS NUMERIC) = c.id
ORDER BY a.JiveLearner.id, b.message.learner_name, question ASC
"""



In [6]:
job_config = bigquery.QueryJobConfig()
now = time.time()
join_query_job = client.query(join_query, location = 'US')
res = join_query_job.result()
print('query took:', round(time.time()-now,2), 's')

query took: 2.45 s


In [7]:
now = time.time()
combined_df = res.to_dataframe()
print('load to pandas from BQ took:', round(time.time()-now, 2),'s')

load to pandas from BQ took: 7.77 s


In [227]:
combined_df[combined_df['answer'] == 'true']

,learner,question,answer,last_modified,name,learner_name,tag,metatag,who,text,answer_type,answer_tag,metatags


In [9]:
combined_true = combined_df[combined_df['answer'] == True]

In [10]:
combined_false = combined_df[combined_df['answer'] == False]

In [143]:
len(set(combined_df['name']))

34355

In [11]:
combined_df.shape

(39441, 13)

In [12]:
#only select the last modified from all of the keys with the same name
combined_df = combined_df.sort_values('last_modified').drop_duplicates(subset='name',keep='last')
combined_df.shape

(34355, 13)

In [13]:
combined_df.columns

Index(['learner', 'question', 'answer', 'last_modified', 'name',
       'learner_name', 'tag', 'metatag', 'who', 'text', 'answer_type',
       'answer_tag', 'metatags'],
      dtype='object')

In [14]:
learner_df = combined_df[['learner']]

In [15]:
learner_df.shape

(34355, 1)

In [16]:
learner_df.reset_index(inplace=True, drop=True)

In [17]:
learner_df = learner_df.drop_duplicates(keep='last')

In [18]:
learner_df.shape

(2967, 1)

In [19]:
#convert to string later for ease of reading?
learner_df = learner_df#.astype(str)

In [20]:
question_df = combined_df[['question']].astype(int)

In [21]:
question_df = question_df.drop_duplicates(keep='last')

In [22]:
#there are fewer questions answered than there are questions in the question database
#question_df represents the number of questions answered in the combined tables
question_df.shape

(803, 1)

In [325]:
question_df

""
question
128
145
138
196
149
206
197
441
339


In [24]:
questions_query="""
#standardSQL
SELECT * FROM `socos-quickstep-601.museathome.questions`
"""

In [25]:
job_config = bigquery.QueryJobConfig()
now = time.time()
questions_query_job = client.query(questions_query, location = 'US')
res = questions_query_job.result()
print('query took:', round(time.time()-now,2), 's')

query took: 0.99 s


In [26]:
now = time.time()
questions_df = res.to_dataframe()
print('load to pandas from BQ took:', round(time.time()-now, 2),'s')

load to pandas from BQ took: 0.66 s


In [27]:
questions_df.shape

(1162, 23)

In [28]:
questions_df = questions_df[['id']].drop_duplicates(keep='last')

In [29]:
questions_df = questions_df.reset_index(drop=True)

In [30]:
dummy = pd.get_dummies(questions_df['id'])

In [31]:
#testing to see if it is better to get questions from combined_df
dummy2 = pd.get_dummies(question_df['question'])

In [32]:
muse_model_df = pd.concat([learner_df, dummy])

In [33]:
#only selects all the users
muse_model_df = muse_model_df[:2967]

In [34]:
muse_model_df = muse_model_df.set_index('learner')

In [100]:
combined_df[combined_df.answer.isnull() & combined_df.answer_tag.isnull()].shape

(47, 13)

In [35]:
combined_df[combined_df['question'] == '367']

,learner,question,answer,last_modified,name,learner_name,tag,metatag,who,text,answer_type,answer_tag,metatags
4667,5130925385973760,367,True,2017-01-15 01:04:01.835620+00:00,qcwDx4KOHCYIAEedUWafUWmqMx0l0Xlg,Bela,[yes],[answer],child,Does {kid_name} attend school outside of the h...,binary,yes,answer
32691,5766696644116480,367,True,2017-01-15 01:07:02.753380+00:00,cx13XwRaIjlAQpyhcagPBYoM4RPMVw8w,Kirsten,[yes],[answer],child,Does {kid_name} attend school outside of the h...,binary,yes,answer
22522,5707635273760768,367,True,2017-01-15 01:08:15.032340+00:00,lNGGV3TIMxxcUH1v3mIYZ7A4D9xSjvPF,Emmett,[yes],[answer],child,Does {kid_name} attend school outside of the h...,binary,yes,answer
3534,5088485891375104,367,True,2017-01-15 01:09:25.820330+00:00,cgUBt9Jtcf7ZhtQkAHyPMwVd8HKAmD5T,Javeria,[yes],[answer],child,Does {kid_name} attend school outside of the h...,binary,yes,answer
10633,5634559156682752,367,True,2017-01-15 01:11:07.749040+00:00,AB0mj3BDXxndfPleozddWHuAsPWoxcx7,BooBoo,[yes],[answer],child,Does {kid_name} attend school outside of the h...,binary,yes,answer
21719,5699788435619840,367,True,2017-01-15 01:11:25.846400+00:00,SwIuWojCRKILpzz4Z0zqv1RQJpbLdcki,Inger,[yes],[answer],child,Does {kid_name} attend school outside of the h...,binary,yes,answer
14657,5661396394049536,367,True,2017-01-15 01:11:33.324540+00:00,jolvaORjGv2ptC67chzCaxdfo6yl6lmc,lilia,[yes],[answer],child,Does {kid_name} attend school outside of the h...,binary,yes,answer
7927,5207672290803712,367,True,2017-01-15 01:11:47.443960+00:00,CQntSTnDPeXmUv97e7Us3Q6YGIMAtLpk,Chris,[yes],[answer],child,Does {kid_name} attend school outside of the h...,binary,yes,answer
21522,5698936018829312,367,True,2017-01-15 01:11:57.539620+00:00,W3ZrH2BPNI40NvhJH06ZayZmx4ssO5fE,Chinmay,[yes],[answer],child,Does {kid_name} attend school outside of the h...,binary,yes,answer
17116,5674390951821312,367,True,2017-01-15 01:12:51.445150+00:00,V9shBEdI0NIxtwz8j4uyTG0u7BR4ScuZ,RaRa,[yes],[answer],child,Does {kid_name} attend school outside of the h...,binary,yes,answer


In [36]:
combined_answertags = combined_df[combined_df['answer_tag'].notnull()]

In [37]:
combined_true.shape

(17316, 13)

In [38]:
combined_false.shape

(7895, 13)

In [102]:
combined_true.shape[0] + combined_false.shape[0] + combined_answertags.shape[0]

41242

In [39]:
# #answers that result in -1
# for i in combined_df.index:
#     if 'Kid is too old' in combined_df.at[i, 'tag'] or \
#         'kid is too old' in combined_df.at[i, 'tag'] or \
#         'kid too old' in combined_df.at[i, 'tag']:
#         learner = combined_df.at[i, 'learner']
#         question = combined_df.at[i, 'question']
#         question = int(question)
#         #print([learner, question])
#         muse_model_df.loc[learner, question] = -1
        
#     if 'Kid is too young' in combined_df.at[i, 'tag'] or \
#     'kid is too young' in combined_df.at[i, 'tag'] or \
#     'kid too young' in combined_df.at[i, 'tag']:
#         learner = combined_df.at[i, 'learner']
#         question = combined_df.at[i, 'question']
#         question = int(question)
#         #print([learner, question])
#         muse_model_df.loc[learner, question] = -1
        
#     if 'doesn\'t apply' in combined_df.at[i, 'tag'] or \
#     '"Doesn\'t apply"' in combined_df.at[i, 'tag']: # or \
# #     'not sure' in combined.at[i, 'tag'] or \
# #     'not yet' in combined.at[i, 'tag']:
#         learner = combined_df.at[i, 'learner']
#         question = combined_df.at[i, 'question']
#         question = int(question)
#         #print(i, [learner, question])
#         muse_model_df.loc[learner, question] = -1

In [40]:
# for i in combined_df.index:
#     learner = combined_df.at[i, 'learner']
#     question = combined_df.at[i, 'question']
#     question = int(question)    
        
#     if 'nope' or 'No' in combined_df.at[i, 'tag']:
#         muse_model_df.loc[learner, question] = -2

#     if 'no' in combined_df.at[i, 'tag']:
#         muse_model_df.loc[learner, question] = -2
    
#     if 'definitely' in combined_df.at[i, 'tag']:
#         muse_model_df.loc[learner, question] = 2
        

In [41]:
# for i in combined_df.index:
#     if combined_df.at[i, 'answer'] == True:
#         learner = combined_df.at[i, 'learner']
#         question = combined_df.at[i, 'question']
#         question = int(question)
#         muse_model_df.loc[learner, question] = 1
        
#     if combined_df.at[i, 'answer'] == False:
#         learner = combined_df.at[i, 'learner']
#         question = combined_df.at[i, 'question']
#         question = int(question)
#         muse_model_df.loc[learner, question] = -1

In [42]:
for i in combined_true.index:
    learner = combined_true.at[i, 'learner']
    question = combined_true.at[i, 'question']
    question = int(question)
    muse_model_df.loc[learner, question] = 1

In [168]:
for i in combined_false.index:
    learner = combined_false.at[i, 'learner']
    question = combined_false.at[i, 'question']
    question = int(question)
    muse_model_df.loc[learner, question] = -1

1.0       1212
-2.0        90
2.0         46
0.0         46
-1.0        29
potato       2
Name: 367, dtype: int64

In [45]:
muse_model_df[367].value_counts()

 1.0    27
-1.0     2
Name: 367, dtype: int64

muse_model_df

In [147]:
muse_model_df.count().sum()

30854

total unique QA pairs = 34355

In [47]:
# for i in combined_answertags.index:
#     learner = combined_answertags.at[i, 'learner']
#     question = combined_answertags.at[i, 'question']
#     question = int(question)
    
#     if 'definitely' or 'of course' or 'absoutely' in combined_answertags.at[i, 'answer_tag']:
#         muse_model_df.loc[learner, question] = 2
        
#     elif 'not sure' or 'Not sure' or 'somewhat' in combined_answertags.at[i, 'answer_tag']:
#         muse_model_df.loc[learner, question] = 0
        
#     elif 'no' or 'nope' or 'No' in combined_answertags.at[i, 'answer_tag']:
#         muse_model_df.loc[learner, question] = -1
        
#     else:
#         skip

#    if 'no' or 'nope' or 'not yet'

adding 'yes' doesn't change the count

In [48]:
# for i in combined_answertags.index:    
#     if 'no' or 'nope' or 'No' in combined_answertags.at[i, 'answer_tag']:
#         learner = combined_answertags.at[i, 'learner']
#         question = combined_answertags.at[i, 'question']
#         question = int(question)
#         muse_model_df.loc[learner, question] = -1

# for i in combined_answertags.index:  
#     if 'definitely' or 'of course' or 'absoutely' in combined_answertags.at[i, 'answer_tag']:
#         learner = combined_answertags.at[i, 'learner']
#         question = combined_answertags.at[i, 'question']
#         question = int(question)
#         muse_model_df.loc[learner, question] = 2
        

In [49]:
muse_model_df[367].value_counts()

 1.0    27
-1.0     2
Name: 367, dtype: int64

In [50]:
muse_model_df[367].value_counts()

 1.0    27
-1.0     2
Name: 367, dtype: int64

In [51]:
muse_model_df[367].value_counts()

 1.0    27
-1.0     2
Name: 367, dtype: int64

In [52]:
muse_model_df.count().sum()

21610

In [126]:
combined_df[combined_df.answer_tag == 'of course'].shape

(75, 13)

In [128]:
combined_df[combined_df.answer_tag == 'dislike'].shape

(0, 13)

In [ ]:
combined_df[combined_df.answer_tag == 'personal'].shape

In [111]:
combined_df[combined_df.answer_tag == 'personal'].shape

(10, 13)

In [53]:
combined_answertags

,learner,question,answer,last_modified,name,learner_name,tag,metatag,who,text,answer_type,answer_tag,metatags
19756,5689015533568000,200,True,2017-01-15 00:56:07.025390+00:00,RjL7yAT8LV381Rr6nxlXiPkoZ8p76gtm,Alchemy,[yes],[answer],child,Does {kid_name} have temper tantrums?,scale,yes,answer
34657,5947893009612800,686,True,2017-01-15 00:56:07.025930+00:00,fQtzzIhJIwaGPCoxPklyfEE8jNS6yR0T,Margo,[yes],[answer],parent,"In the last week, have you spent time playing ...",scale,yes,answer
1761,4920672459423744,891,False,2017-01-15 00:56:07.026480+00:00,01C3KSV4fkBSdjoelJSc3LgQUuVxGicK,Sam,[no],[answer],parent,Let’s assume a tax accountant and an artist ma...,scale,no,answer
17035,5673490367643648,773,True,2017-01-15 00:56:07.027430+00:00,dHDKkxi9v9TO7fK9y9XsLDd7pq9694JS,Oliver,[yes],[answer],child,"In the past week, has {kid_name} lost {their} ...",scale,yes,answer
25266,5723846241943552,600,False,2017-01-15 00:56:07.030410+00:00,OE4MHZFRJz8NefIWu2BXkdYmLZ4NZAHL,Hazel,[no],[answer],parent,Do you want {kid_name} to do the same activiti...,scale,no,answer
24298,5718264999051264,605,True,2017-01-15 00:56:07.032790+00:00,IWlQePJji4R1nhHT644SPWRVAQz7P5n2,Elisa,[yes],[answer],child,Does {kid_name} have a hard time adjusting to ...,scale,yes,answer
8944,5462642772672512,288,True,2017-01-15 00:56:07.033070+00:00,J4uNcMlvm5gr99IJWpRHshoWR2DoZgp1,Annika,[yes],[answer],child,Does {kid_name} become frustrated when other c...,scale,yes,answer
2231,4999681536950272,785,True,2017-01-15 00:56:07.033510+00:00,2Sn46LhrMJf1SN0XwmW4b7h7FBBV5bnk,Rowan,[yes],[answer],parent,Are you interested in learning about the peopl...,scale,yes,answer
8931,5462642772672512,270,True,2017-01-15 00:56:07.038690+00:00,Y76V1uck3XPdAUmdn14jNu3oZOeI2sq2,Annika,[yes],[answer],child,Does {kid_name} know how to turn on/off lights?,scale,yes,answer
24368,5718264999051264,926,False,2017-01-15 00:56:07.039030+00:00,UIv7N2akpuMm2MVJOCdOwOrbba4PCQhr,Elisa,[no],[answer],child,Is your child enrolled in a free lunch program...,binary,no,answer


In [263]:
count_no_answer = 0
feedback_list = []
for i in combined_answertags.index:
    if 'answer' not in combined_answertags.at[i, 'metatag']:
        feedback_list.append(combined_answertags.at[i, 'answer_tag'])
        count_no_answer += 1

In [260]:
count_no_answer

468

In [258]:
answers

15563

In [270]:
feedback_dict = {'Kid is too young, Kid is too old': 0,
 'absolutely': 0,
 'absolutely, good idea': 0,
 'absolutely, of course': 0,
 'abstract': 0,
 "doesn't apply": 0,
 "doesn't apply, abstract": 0,
 "doesn't apply, kid too young": 0,
 'good idea': 0,
 'interesting': 0,
 'kid answered, good idea': 0,
 'kid answered, of course, abstract': 0,
 'kid too old, personal, interesting': 0,
 'kid too young': 0,
 "kid too young, doesn't apply": 0,
 "kid too young, doesn't apply, not yet, kid answered": 0,
 'kid too young, not yet': 0,
 'not yet': 0,
 "not yet, doesn't apply, kid too young": 0,
 'not yet, kid too young': 0,
 'of course': 0,
 'of course, not yet, philosophical': 0,
 'personal': 0}

In [272]:
for each in feedback_list:
    if each in feedback_dict:
        feedback_dict[each] += 1

In [273]:
feedback_dict

{'Kid is too young, Kid is too old': 1,
 'absolutely': 66,
 'absolutely, good idea': 1,
 'absolutely, of course': 3,
 'abstract': 2,
 "doesn't apply": 87,
 "doesn't apply, abstract": 1,
 "doesn't apply, kid too young": 1,
 'good idea': 4,
 'interesting': 2,
 'kid answered, good idea': 1,
 'kid answered, of course, abstract': 1,
 'kid too old, personal, interesting': 1,
 'kid too young': 89,
 "kid too young, doesn't apply": 1,
 "kid too young, doesn't apply, not yet, kid answered": 1,
 'kid too young, not yet': 4,
 'not yet': 110,
 "not yet, doesn't apply, kid too young": 1,
 'not yet, kid too young': 5,
 'of course': 75,
 'of course, not yet, philosophical': 1,
 'personal': 10}

In [267]:

set(feedback_list)

{'Kid is too young, Kid is too old',
 'absolutely',
 'absolutely, good idea',
 'absolutely, of course',
 'abstract',
 "doesn't apply",
 "doesn't apply, abstract",
 "doesn't apply, kid too young",
 'good idea',
 'interesting',
 'kid answered, good idea',
 'kid answered, of course, abstract',
 'kid too old, personal, interesting',
 'kid too young',
 "kid too young, doesn't apply",
 "kid too young, doesn't apply, not yet, kid answered",
 'kid too young, not yet',
 'not yet',
 "not yet, doesn't apply, kid too young",
 'not yet, kid too young',
 'of course',
 'of course, not yet, philosophical',
 'personal'}

In [317]:
for i in combined_answertags.index:
    if 'answer' in combined_answertags.at[i, 'metatag']:
        combined_answertags.loc[i, 'final_answer'] = combined_answertags.at[i, 'tag'][0]

KeyboardInterrupt: 

In [318]:
for i in combined_answertags.index:
    if 'answer' in combined_answertags.at[i, 'metatag']:
        combined_answertags.at[i, 'final_answer2'] = combined_answertags.at[i, 'tag'][0]

In [319]:
combined_answertags

,learner,question,answer,last_modified,name,learner_name,tag,metatag,who,text,answer_type,answer_tag,metatags,final_answer,final_answer2
19756,5689015533568000,200,True,2017-01-15 00:56:07.025390+00:00,RjL7yAT8LV381Rr6nxlXiPkoZ8p76gtm,Alchemy,[yes],[answer],child,Does {kid_name} have temper tantrums?,scale,yes,answer,yes,yes
34657,5947893009612800,686,True,2017-01-15 00:56:07.025930+00:00,fQtzzIhJIwaGPCoxPklyfEE8jNS6yR0T,Margo,[yes],[answer],parent,"In the last week, have you spent time playing ...",scale,yes,answer,yes,yes
1761,4920672459423744,891,False,2017-01-15 00:56:07.026480+00:00,01C3KSV4fkBSdjoelJSc3LgQUuVxGicK,Sam,[no],[answer],parent,Let’s assume a tax accountant and an artist ma...,scale,no,answer,no,no
17035,5673490367643648,773,True,2017-01-15 00:56:07.027430+00:00,dHDKkxi9v9TO7fK9y9XsLDd7pq9694JS,Oliver,[yes],[answer],child,"In the past week, has {kid_name} lost {their} ...",scale,yes,answer,yes,yes
25266,5723846241943552,600,False,2017-01-15 00:56:07.030410+00:00,OE4MHZFRJz8NefIWu2BXkdYmLZ4NZAHL,Hazel,[no],[answer],parent,Do you want {kid_name} to do the same activiti...,scale,no,answer,no,no
24298,5718264999051264,605,True,2017-01-15 00:56:07.032790+00:00,IWlQePJji4R1nhHT644SPWRVAQz7P5n2,Elisa,[yes],[answer],child,Does {kid_name} have a hard time adjusting to ...,scale,yes,answer,yes,yes
8944,5462642772672512,288,True,2017-01-15 00:56:07.033070+00:00,J4uNcMlvm5gr99IJWpRHshoWR2DoZgp1,Annika,[yes],[answer],child,Does {kid_name} become frustrated when other c...,scale,yes,answer,yes,yes
2231,4999681536950272,785,True,2017-01-15 00:56:07.033510+00:00,2Sn46LhrMJf1SN0XwmW4b7h7FBBV5bnk,Rowan,[yes],[answer],parent,Are you interested in learning about the peopl...,scale,yes,answer,yes,yes
8931,5462642772672512,270,True,2017-01-15 00:56:07.038690+00:00,Y76V1uck3XPdAUmdn14jNu3oZOeI2sq2,Annika,[yes],[answer],child,Does {kid_name} know how to turn on/off lights?,scale,yes,answer,yes,yes
24368,5718264999051264,926,False,2017-01-15 00:56:07.039030+00:00,UIv7N2akpuMm2MVJOCdOwOrbba4PCQhr,Elisa,[no],[answer],child,Is your child enrolled in a free lunch program...,binary,no,answer,no,no


In [299]:
combined_answertags[combined_answertags.final_answer.isnull() & combined_answertags.answer.isnull()]

,learner,question,answer,last_modified,name,learner_name,tag,metatag,who,text,answer_type,answer_tag,metatags,final_answer
11349,5638203017003008,800,None,2017-01-15 01:11:38.423270+00:00,AXsbIXeqOkJOnET7GdgquSvFYhN4VZ93,Hezekiah,[doesn't apply],[feedback],parent,Do you get frustrated if you are forced to exp...,scale,doesn't apply,feedback,NaN
21855,5701810610241536,603,None,2017-01-15 01:16:56.132170+00:00,eQm0lvD2gW1C9loOxEvPfkUwFcy67x9p,Kiddo,"[Kid is too young, Kid is too old]","[feedback, feedback]",parent,Do you believe that money is actually the secr...,scale,"Kid is too young, Kid is too old","feedback, feedback",NaN
15406,5665497936822272,391,None,2017-02-10 04:57:20.671430+00:00,4HCbOXmdybSyGxMpVXACeXLhNKYGsduF,Teagan,[good idea],[feedback],parent,Do you enjoy work that requires you to think h...,scale,good idea,feedback,NaN
15396,5665497936822272,39,None,2017-02-18 01:34:16.339660+00:00,UYVHOZTjmu4YLteMth9xPT87yzeQ36gC,Teagan,"[kid too young, doesn't apply, not yet, kid an...","[feedback, feedback, feedback, feedback]",parent,Did you graduate from college?,binary,"kid too young, doesn't apply, not yet, kid ans...","feedback, feedback, feedback, feedback",NaN
15594,5665497936822272,699,None,2017-03-09 04:51:32.979910+00:00,PBbxEEAVNLycCjrF7c0F2Dchs9D6wSbI,Teagan,"[kid answered, good idea]","[feedback, feedback]",parent,"In the last week, have you had a hard time bal...",scale,"kid answered, good idea","feedback, feedback",NaN
1936,4929646726479872,192,None,2017-04-03 14:39:57.218070+00:00,Qvt7ozJODXGZKPbrCvE85yl9pG8cBSaa,Micah,[not yet],[feedback],child,Do you and {kid_name} brush {their} teeth toge...,scale,not yet,feedback,NaN
22268,5706522961117184,18,None,2017-04-10 00:23:06.557220+00:00,H4ByhDc1DXHE9jFDE1xE6kRnOpL8BbGI,Alexander,[not yet],[feedback],parent,Does your spouse work multiple jobs?,binary,not yet,feedback,NaN
7784,5205634144272384,40,None,2017-04-11 00:26:43.462810+00:00,syoVxZYG9gFmOmn30AWsVT6zxeM2gNet,Mina,[not yet],[feedback],parent,Do you have a graduate degree?,binary,not yet,feedback,NaN
14031,5656462332264448,35,None,2017-04-26 18:06:46.240900+00:00,KRG6odqVWZV43yJvHsDeEgnKFZoMsFDN,Adeline,[not yet],[feedback],child,Does {kid_name} have a close friend outside of...,binary,not yet,feedback,NaN
20494,5691127080419328,39,None,2017-05-13 15:40:11.746270+00:00,OSIqqtrsrqy2vYqN4XP94MeBTVG0IjEt,Jocelyn,[of course],[feedback],parent,Did you graduate from college?,binary,of course,feedback,NaN


In [306]:
combined_tags_final = combined_answertags[combined_answertags.final_answer.notnull() & combined_answertags.answer.isnull()]

In [307]:
combined_tags_final

,learner,question,answer,last_modified,name,learner_name,tag,metatag,who,text,answer_type,answer_tag,metatags,final_answer
30205,5751903552012288,13,None,2017-01-15 00:56:24.831300+00:00,c5HaCuJlA2ys7J7M0tm5rcsvVajTH3dF,Kahlen,[Yes],[answer],parent,Do you know your next door neighbor's first name?,binary,Yes,answer,Yes
27114,5735286357098496,661,None,2017-01-15 00:59:03.526600+00:00,nPBnBlSER4133ZOWYS5hl9mSTscXcJ4y,Junior,[Yes],[answer],parent,"When work becomes very busy, do you notice you...",scale,Yes,answer,Yes
18922,5685665089978368,845,None,2017-01-15 01:00:43.139610+00:00,LpfbNDlL4CDyvcF0sZBN91raPTwsX9v0,Hazel,[yes],[answer],parent,Would you consider a daycare provider who is c...,scale,yes,answer,yes
33051,5767494685949952,16,None,2017-01-15 01:02:31.825530+00:00,loJ4ZvoDRfk5K3dzUeipk2k0fMBKtQUa,Thumper,[barely],[answer],parent,Do you have a hard time maintaining focus if a...,scale,barely,answer,barely
33058,5767494685949952,21,None,2017-01-15 01:03:46.226560+00:00,GJxFu2uWwJoUqjGkUYC2oSPOcazVcllM,Thumper,[yes],[answer],parent,Do you know the name of {kid_name}'s teacher?,scale,yes,answer,yes
12893,5649952705347584,22,None,2017-01-15 01:04:09.841000+00:00,kwvOsDY0i9KQAtVsEjicTRx4xFsodhFj,Tigger,[yes],[answer],parent,Do you know where the nearest playground is fr...,binary,yes,answer,yes
33220,5767494685949952,485,None,2017-01-15 01:04:48.837530+00:00,bB98D8DKKXju1eIzUwuY9ik8OPgMPtcz,Thumper,[no],[answer],parent,"In the last year, has a personal relationship ...",scale,no,answer,no
36589,6231905506689024,927,None,2017-01-15 01:04:56.542510+00:00,czv3R7dnua1YIWYh9lw3GnkDdrDMGDIt,Hamilton,[Yes],[answer],child,If {kid_name} sees {their} siblings struggling...,scale,Yes,answer,Yes
18921,5685665089978368,845,None,2017-01-15 01:05:19.440340+00:00,0jjX5uzhZHN80u4rU0YhX4e0wvmXWGl7,Hazel,[yes],[answer],parent,Would you consider a daycare provider who is c...,scale,yes,answer,yes
30419,5751903552012288,682,None,2017-01-15 01:05:26.753340+00:00,LCLSkED3HJgxtkkzq3RuPWHIG1GxavBU,Kahlen,[Yes],[answer],parent,Do you and {kid_name} often spend time togethe...,scale,Yes,answer,Yes


In [294]:
combined_answertags

,learner,question,answer,last_modified,name,learner_name,tag,metatag,who,text,answer_type,answer_tag,metatags,final_answer
19756,5689015533568000,200,True,2017-01-15 00:56:07.025390+00:00,RjL7yAT8LV381Rr6nxlXiPkoZ8p76gtm,Alchemy,[yes],[answer],child,Does {kid_name} have temper tantrums?,scale,yes,answer,yes
34657,5947893009612800,686,True,2017-01-15 00:56:07.025930+00:00,fQtzzIhJIwaGPCoxPklyfEE8jNS6yR0T,Margo,[yes],[answer],parent,"In the last week, have you spent time playing ...",scale,yes,answer,yes
1761,4920672459423744,891,False,2017-01-15 00:56:07.026480+00:00,01C3KSV4fkBSdjoelJSc3LgQUuVxGicK,Sam,[no],[answer],parent,Let’s assume a tax accountant and an artist ma...,scale,no,answer,no
17035,5673490367643648,773,True,2017-01-15 00:56:07.027430+00:00,dHDKkxi9v9TO7fK9y9XsLDd7pq9694JS,Oliver,[yes],[answer],child,"In the past week, has {kid_name} lost {their} ...",scale,yes,answer,yes
25266,5723846241943552,600,False,2017-01-15 00:56:07.030410+00:00,OE4MHZFRJz8NefIWu2BXkdYmLZ4NZAHL,Hazel,[no],[answer],parent,Do you want {kid_name} to do the same activiti...,scale,no,answer,no
24298,5718264999051264,605,True,2017-01-15 00:56:07.032790+00:00,IWlQePJji4R1nhHT644SPWRVAQz7P5n2,Elisa,[yes],[answer],child,Does {kid_name} have a hard time adjusting to ...,scale,yes,answer,yes
8944,5462642772672512,288,True,2017-01-15 00:56:07.033070+00:00,J4uNcMlvm5gr99IJWpRHshoWR2DoZgp1,Annika,[yes],[answer],child,Does {kid_name} become frustrated when other c...,scale,yes,answer,yes
2231,4999681536950272,785,True,2017-01-15 00:56:07.033510+00:00,2Sn46LhrMJf1SN0XwmW4b7h7FBBV5bnk,Rowan,[yes],[answer],parent,Are you interested in learning about the peopl...,scale,yes,answer,yes
8931,5462642772672512,270,True,2017-01-15 00:56:07.038690+00:00,Y76V1uck3XPdAUmdn14jNu3oZOeI2sq2,Annika,[yes],[answer],child,Does {kid_name} know how to turn on/off lights?,scale,yes,answer,yes
24368,5718264999051264,926,False,2017-01-15 00:56:07.039030+00:00,UIv7N2akpuMm2MVJOCdOwOrbba4PCQhr,Elisa,[no],[answer],child,Is your child enrolled in a free lunch program...,binary,no,answer,no


In [54]:
categories = list(set(combined_answertags['answer_tag']))

In [255]:
(combined_answertags['answer_tag']).value_counts().sum()

16031

In [253]:
(combined_answertags['answer_tag']).value_counts()

yes                                                                 9013
no                                                                  2906
definitely                                                          1850
nope                                                                 685
yes, of course                                                       138
somewhat                                                             137
not yet                                                              110
yes, absolutely                                                      105
probably                                                              91
kid too young                                                         89
doesn't apply                                                         87
of course                                                             75
absolutely                                                            66
barely                                             

In [249]:
categories

["not yet, personal, philosophical, interesting, no, doesn't apply, good idea",
 'nope, not yet',
 "not yet, doesn't apply, kid too young",
 'good idea, no, kid too young, interesting',
 'definitely, kid too young',
 'yes, of course, personal',
 'no, good idea, not yet, kid too young',
 'yes, abstract',
 'no, not yet, kid too young',
 'no, good idea, interesting',
 'definitely, kid answered',
 'definitely, absolutely, of course, good idea',
 "probably, abstract, doesn't apply, personal",
 'yes, absolutely, of course',
 'absolutely, definitely',
 'no, philosophical',
 'not yet, no',
 'yes, interesting, philosophical',
 'no, good idea',
 'kid too young',
 'nope, not yet, somewhat, kid answered, kid too old',
 "definitely, doesn't apply",
 "kid too old, personal, philosophical, kid too young, of course, absolutely, good idea, interesting, kid answered, doesn't apply, not yet, abstract, yes",
 "Yes, Doesn't apply, No, Not sure",
 'kid too young, no, not yet',
 'kid answered, yes',
 'no, ki

In [136]:
len(set(combined_answertags.name) - set(combined_true.name) - set(combined_false.name))

9824

In [141]:
len(set(combined_false.name))

7781

In [138]:
len(set(combined_true.name) - set(combined_false.name))

16704

In [129]:
combined_answertags['answer_tag'].value_counts().sum()

16031

In [56]:
# def process_df(df):
#     for i in combined_answertags.index:
#         learner = combined_answertags.at[i, 'learner']
#         question = combined_answertags.at[i, 'question']
#         question = int(question)    

#         if 'of course' in combined_answertags.at[i, 'tag']:
#             muse_model_df.loc[learner, question] = 2

#         if 'definitely' in combined_answertags.at[i, 'tag']:
#             muse_model_df.loc[learner, question] = 2

#         if 'absolutely' in combined_answertags.at[i, 'tag']:
#             muse_model_df.loc[learner, question] = 2

#         if 'yes' in combined_answertags.at[i, 'tag']:
#             muse_model_df.loc[learner, question] = 1

#         if 'Yes' in combined_answertags.at[i, 'tag']:
#             muse_model_df.loc[learner, question] = 1

#         if 'nope' in combined_answertags.at[i, 'tag']:
#             muse_model_df.loc[learner, question] = -2

#         if 'no' in combined_answertags.at[i, 'tag']:
#             muse_model_df.loc[learner, question] = -2
            
#         return(df)

In [ ]:
#     if answer == 'yes':
#         muse_model_df.loc[learner, question] = 1
        
#     if answer == 'no':
#         muse_model_df.loc[learner, question] = -1
        
#     if answer == 'definitely':
#         muse_model_df.loc[learner, question] = 2
        
#     if answer == 'probably':
#         muse_model_df.loc[learner, question] = 1
         

In [280]:
combined_answertags.at[1, 'tag'][0].lower()

'yes'

In [ ]:
answers: {
        'yes': 1,
        'no': -1,
        'definitely': 2,
        'probably': 1,
        'somewhat': 0,
        'barely': -1,
        'nope': -2
    }
        

In [313]:
def switcher(ans):
    answers = {
        'yes': 1,
        'no': -1,
        'definitely': 2,
        'probably': 1,
        'somewhat': 0,
        'barely': -1,
        'nope': -2
    }
    
    return answers.get(ans, None)

def add_tags(df):
    for i in df.index:    
        answer = df.at[i, 'final_answer']
        learner = df.at[i, 'learner']
        question = df.at[i, 'question']
        question = int(question) 
        
        muse_answer = switcher(answer)
        muse_model_df.loc[learner, question] = muse_answer

In [314]:
add_tags(combined_tags_final)

In [315]:
print(muse_model_df.count().sum())

45583


In [316]:
muse_model_df

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,...,1059,1097,1099,10,2,481,502,460,482,933
learner,,,,,,,,,,,,,,,,,,,,,
6051420109275136,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5134423787831296,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5798113272922112,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5756901417549824,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5630121163620352,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5628773986729984,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5637257553772544,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5722679185244160,None,None,None,None,None,None,1,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6125581443792896,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [234]:
for i in combined_answertags.index:
    learner = combined_answertags.at[i, 'learner']
    question = combined_answertags.at[i, 'question']
    question = int(question)    
        
    if 'of course' in combined_answertags.at[i, 'answer_tag']:
        muse_model_df.loc[learner, question] = 2
        
    if 'definitely' in combined_answertags.at[i, 'answer_tag']:
        muse_model_df.loc[learner, question] = 2
        
    if 'absolutely' in combined_answertags.at[i, 'answer_tag']:
        muse_model_df.loc[learner, question] = 2
        
    if 'yes' in combined_answertags.at[i, 'answer_tag']:
        muse_model_df.loc[learner, question] = 1
    
    #capital Y
    if 'Yes' in combined_answertags.at[i, 'answer_tag']:
        muse_model_df.loc[learner, question] = 1
        
    if 'probably' in combined_answertags.at[i, 'answer_tag']:
        muse_model_df.loc[learner, question] = 1
        
    if 'somewhat' in combined_answertags.at[i, 'answer_tag']:
        muse_model_df.loc[learner, question] = 0

    #Doesn't apply, doesn't apply...
    if 'apply' in combined_answertags.at[i, 'answer_tag']:
        muse_model_df.loc[learner, question] = 0
        
    #too young, too old..
    if 'too' in combined_answertags.at[i, 'answer_tag']:
        muse_model_df.loc[learner, question] = 0
        
    if 'barely' in combined_answertags.at[i, 'answer_tag']:
        muse_model_df.loc[learner, question] = -1
    
    #not sure, not yet..
    if 'not' in combined_answertags.at[i, 'answer_tag']:
        muse_model_df.loc[learner, question] = -1
    
    #capital N
    if 'Not' in combined_answertags.at[i, 'answer_tag']:
        muse_model_df.loc[learner, question] = -1
        
    if 'nope' in combined_answertags.at[i, 'answer_tag']:
        muse_model_df.loc[learner, question] = -2
        
    #overwrite the 'no' in 'not' mistakes
    if 'no' in combined_answertags.at[i, 'tag']:
        muse_model_df.loc[learner, question] = -2
        
    if 'No' in combined_answertags.at[i, 'tag']:
        muse_model_df.loc[learner, question] = -2
        
#     else:
#         print(i, question, learner)

19756 200 5689015533568000
34657 686 5947893009612800
1761 891 4920672459423744
17035 773 5673490367643648
25266 600 5723846241943552
24298 605 5718264999051264
8944 288 5462642772672512
2231 785 4999681536950272
8931 270 5462642772672512
24368 926 5718264999051264
15279 140 5665497936822272
25141 691 5723548412805120
31286 50 5755655105282048
3300 154 5082384487677952
10551 844 5633772657573888
16865 715 5672256034308096
25106 25 5723548412805120
14521 739 5658145321910272
27336 780 5735545934184448
25269 676 5723846241943552
21094 27 5695127876009984
30205 13 5751903552012288
1670 776 4915599499067392
14475 914 5657375784566784
8076 265 5271646650761216
14179 801 5656462332264448
24962 610 5722522989363200
3302 159 5082384487677952
28959 770 5744541323755520
2497 412 5066257690787840
22380 843 5706522961117184
21611 790 5699450357940224
30977 263 5754323531202560
29185 621 5746784991182848
37145 317 6262400311361536
24328 711 5718264999051264
6405 581 5170979328229376
37785 740 62980

17458 390 5677691583856640
18823 395 5685665089978368
13796 576 5655148575588352
28669 795 5743068233859072
10362 685 5633067678957568
27683 803 5737552623435776
31413 329 5756346527907840
32575 569 5765283616653312
5386 691 5143752456798208
15887 49 5665731089793024
35689 277 6057041550376960
31439 891 5756346527907840
35088 689 5989180261793792
20751 470 5693766841139200
1815 410 4921226711531520
20756 516 5693766841139200
3099 391 5078095828615168
13187 650 5649952705347584
26300 41 5731400284110848
2220 708 4999681536950272
30661 698 5753217711341568
16822 23 5672256034308096
25860 760 5727790498316288
25463 537 5725996342837248
33585 201 5768372503445504
3837 262 5104547995320320
18143 177 5682510201618432
16808 631 5672142083457024
28951 600 5744541323755520
30886 773 5753938796085248
10192 406 5632145267621888
13855 852 5655148575588352
32187 890 5762578928107520
37672 23 6298093637074944
13830 716 5655148575588352
25112 393 5723548412805120
32639 879 5765283616653312
6731 845 5

27535 485 5736949784510464
15481 568 5665497936822272
10270 841 5632145267621888
11433 9 5638203017003008
5830 789 5156636687597568
38899 267 6534978800189440
28643 662 5743068233859072
19758 211 5689015533568000
2190 588 4999681536950272
7386 569 5191211040112640
13259 812 5649952705347584
16177 721 5667017885483008
25098 221 5723548412805120
4729 827 5132086318989312
11118 108 5638203017003008
28638 596 5743068233859072
28650 680 5743068233859072
35791 847 6057041550376960
21095 27 5695127876009984
1890 819 4921226711531520
8985 503 5462642772672512
32152 699 5762578928107520
21168 785 5695127876009984
25919 494 5728512723910656
22253 153 5706522961117184
10539 811 5633772657573888
24318 689 5718264999051264
25829 487 5727790498316288
32640 889 5765283616653312
5221 903 5140419025305600
14469 899 5657375784566784
29303 840 5747175027900416
28972 887 5744541323755520
5219 900 5140419025305600
11633 685 5638680731451392
15507 578 5665497936822272
20804 916 5693766841139200
23074 794 57

8048 219 5271646650761216
21961 568 5703462461374464
3928 697 5104547995320320
3301 157 5082384487677952
34891 529 5980925334650880
15689 833 5665497936822272
1874 717 4921226711531520
27622 409 5737552623435776
25148 775 5723548412805120
31509 766 5756614627819520
28906 110 5744541323755520
30973 228 5754323531202560
228 357 4649666666823680
24321 689 5718264999051264
14037 376 5656462332264448
25144 744 5723548412805120
28608 355 5743068233859072
20744 383 5693766841139200
14171 712 5656462332264448
9748 741 5630088112504832
29288 68 5747175027900416
6648 812 5179196942843904
10277 863 5632145267621888
34698 834 5947893009612800
24572 371 5720785188552704
20202 711 5689894156369920
15746 930 5665497936822272
3971 98 5104547995320320
18737 13 5685665089978368
31314 602 5755655105282048
204 230 4649666666823680
22188 853 5705584510763008
8149 645 5271646650761216
23686 18 5716507619229696
16812 868 5672142083457024
30903 849 5753938796085248
30659 668 5753217711341568
31194 107 5755655

18475 614 5683762654347264
33715 790 5768584097693696
9709 1 5630088112504832
12264 406 5643814291111936
7551 40 5196373121040384
13303 914 5649952705347584
21475 389 5698568648130560
22522 367 5707635273760768
11153 190 5638203017003008
30656 601 5753217711341568
29182 595 5746784991182848
11157 199 5638203017003008
18110 484 5682414571487232
33687 274 5768584097693696
7794 462 5205634144272384
24365 911 5718264999051264
20586 81 5691830263873536
16844 581 5672256034308096
31421 429 5756346527907840
29256 401 5747175027900416
22384 851 5706522961117184
24910 14 5722522989363200
8106 421 5271646650761216
5785 393 5156636687597568
2099 157 4999681536950272
27319 695 5735545934184448
19789 717 5689015533568000
8999 567 5462642772672512
34667 712 5947893009612800
9070 830 5462642772672512
12788 813 5648633076645888
28605 320 5743068233859072
15155 658 5663804612411392
15891 68 5665731089793024
28632 549 5743068233859072
15864 15 5665731089793024
33686 269 5768584097693696
3894 575 5104547

...

23563 368 5714492050636800
23564 371 5714492050636800
9148 368 5473750543761408
9149 371 5473750543761408
14621 367 5660191047548928
14622 368 5660191047548928
33995 371 5816560337813504
33991 367 5816560337813504
9239 35 5508004308844544
9243 42 5508004308844544
21397 367 5697669469044736
21398 371 5697669469044736
4984 741 5138798593703936
13543 5 5651514844512256
7978 371 5262536232402944
7977 368 5262536232402944
1154 25 4870873999736832
1148 1066 4870873999736832
1173 5 4870873999736832
19967 18 5689698689220608
19972 926 5689698689220608
6930 367 5184155025408000
6931 371 5184155025408000
29251 367 5747104978829312
17316 367 5675596826804224
4794 367 5135497038921728
4795 371 5135497038921728
33495 590 5767750609797120
3764 847 5102615138402304
4229 371 5115208922038272
4228 368 5115208922038272
18016 367 5680447086395392
18017 371 5680447086395392
18513 368 5683865611927552
18514 371 5683865611927552
1547 43 4913639731494912
34832 368 5978992477732864
19113 368 5686223112765440


3488 40 5085518752645120
20309 367 5690692194009088
33943 367 5769738852499456
33945 368 5769738852499456
31405 368 5756319294291968
31406 371 5756319294291968
110 1099 4570824568733696
8690 368 5384744728526848
5446 371 5143870446764032
19966 15 5689698689220608
11661 368 5639006087806976
11662 371 5639006087806976
12373 741 5644012383895552
12369 35 5644012383895552
10575 58 5633877192212480
603 25 4776652622004224
5631 22 5154666249715712
2700 367 5068206678999040
2701 371 5068206678999040
2722 367 5069831955021824
2723 371 5069831955021824
24657 1097 5721187929817088
24658 35 5721187929817088
20691 38 5693490809798656
20696 590 5693490809798656
20695 43 5693490809798656
20693 4 5693490809798656
20692 38 5693490809798656
12406 5 5644487137165312
11967 41 5641599576965120
36661 58 6241349690982400
22618 590 5708405721268224
31441 367 5756392409399296
31445 368 5756392409399296
28326 368 5741631378554880
28327 371 5741631378554880
18565 741 5684914590580736
20270 368 5690095461990400


12363 368 5643895975182336
12365 371 5643895975182336
26841 368 5733472886849536
26842 371 5733472886849536
19021 22 5686000604938240
19020 15 5686000604938240
13574 18 5652075874615296
3487 39 5085518752645120
31070 1097 5754726096306176
188 43 4643216242507776
34957 15 5982813690003456
3252 24 5079653211439104
509 590 4728182171238400
3559 368 5089350433898496
3560 371 5089350433898496
32015 24 5761834132963328
34463 368 5911510756884480
34464 371 5911510756884480
3719 368 5097610494869504
3720 371 5097610494869504
19998 368 5689783070228480
19999 371 5689783070228480
2339 2 5038501477220352
2080 22 4975444847755264
35949 5 6077804105957376
22111 42 5705113024856064
608 4 4776652622004224
6912 590 5183241640214528
8012 2 5269789098377216
23120 2 5712455992868864
18287 13 5682720877314048
18292 38 5682720877314048
22536 368 5707731738558464
22537 371 5707731738558464
31087 13 5755045333172224
12811 38 5649052611903488
1285 2 4881077638266880
8481 847 5362613902901248
27813 367 5737927

11739 371 5639581311434752
29435 18 5747607347396608
22114 25 5705205626699776
37020 1097 6254170495516672
37029 42 6254170495516672
34838 38 5978992477732864
19103 41 5686198345400320
21260 43 5696111368994816
124 368 4613797645910016
125 371 4613797645910016
38078 39 6317995286593536
38075 15 6317995286593536
38074 10 6317995286593536
18406 58 5683725555728384
7638 38 5198826902126592
5227 10 5140608351993856
24407 25 5719302212681728
36049 42 6141755665678336
25066 40 5723352664637440
25057 15 5723352664637440
25067 5 5723352664637440
25069 58 5723352664637440
6286 368 5169309051518976
6287 371 5169309051518976
8011 15 5269789098377216
22521 40 5707542458007552
1044 43 4845229765033984
35612 367 6038306496184320
22117 367 5705223460880384
5239 847 5141049768935424
22118 371 5705223460880384
24410 40 5719302212681728
35328 10 6021337030066176
12803 13 5649052611903488
12616 38 5647843935125504
12623 39 5647843935125504
12634 5 5647843935125504
4954 18 5138335722897408
4967 42 5138335

7039 367 5186049156317184
7040 371 5186049156317184
13499 5 5651098123632640
13493 40 5651098123632640
14615 42 5659784879538176
32684 367 5766526267293696
10680 590 5634932344881152
23792 847 5717123137536000
23789 13 5717123137536000
25339 41 5725205729116160
32208 40 5762892506857472
21410 367 5698197062156288
21411 371 5698197062156288
7323 58 5189887531679744
1286 22 4881077638266880
6555 43 5177740961513472
13595 367 5652468042039296
13596 371 5652468042039296
12506 367 5646070621143040
12507 371 5646070621143040
5474 41 5145024048136192
24187 24 5718042545750016
14609 25 5659784879538176
22094 22 5704698015252480
39123 40 6562469224906752
16793 367 5671751774109696
16794 371 5671751774109696
13433 367 5650886118342656
13435 371 5650886118342656
37538 2 6283284199768064
24444 1066 5720141639712768
36041 1097 6135557054791680
35242 24 6005362368970752
2411 10 5052648327741440
29876 847 5750196516421632
2412 25 5052648327741440
30540 368 5752751367651328
30541 371 5752751367651328


In [ ]:
def numbers_to_strings(argument): 
    switcher = { 
         #'Dislike': None,
         "Doesn't apply": 0,
         'Kid is too old': -1,
         'Kid is too young': -1,
         'No': -1,
         'Not sure': 0,
         'Yes': 1,
         'absolutely': 2,
         #'abstract': None,
         'barely': -1,
         'definitely': 2,
         "doesn't apply": 0,
         #'good idea': None,
         #'interesting',
         #'kid answered',
         'kid is too old': 0,
         'kid is too young': 0,
         'kid too old': 0,
         'kid too young': 0,
         'no',
         'nope',
         'not sure',
         'not yet',
         'of course',
         'personal',
         'philosophical',
         'probably',
         'somewhat',
         'yes'        
        
    } 
  
    # get() method of dictionary data type returns  
    # value of passed argument if it is present  
    # in dictionary otherwise second argument will 
    # be assigned as default value of passed argument 
    return switcher.get(argument, None) 

In [233]:
combined_answertags[combined_answertags['learner'] == 5661396394049536]

,learner,question,answer,last_modified,name,learner_name,tag,metatag,who,text,answer_type,answer_tag,metatags
14658,5661396394049536,369,True,2017-01-15 01:02:23.563940+00:00,RxrKlc86i8T6cNZzj5LjwRFsiCNVVYQu,lilia,[yes],[answer],parent,Do you read with {kid_name}?,scale,yes,answer
14657,5661396394049536,367,True,2017-01-15 01:11:33.324540+00:00,jolvaORjGv2ptC67chzCaxdfo6yl6lmc,lilia,[yes],[answer],child,Does {kid_name} attend school outside of the h...,binary,yes,answer
14659,5661396394049536,371,False,2017-01-15 01:16:56.135220+00:00,KMs6Qy97Da53zbFmg5eUUVCIe4jwLcQ8,lilia,[no],[answer],parent,Do you work outside of the home?,binary,no,answer


In [243]:
muse_model_df.loc[5661396394049536, 'q369']

1.0

In [ ]:
if 'yes' in combined_answertag.at[]

In [239]:
combined_answertags.loc[19756, :]

learner                               5689015533568000
question                                           200
answer                                            True
last_modified         2017-01-15 00:56:07.025390+00:00
name                  RjL7yAT8LV381Rr6nxlXiPkoZ8p76gtm
learner_name                                   Alchemy
tag                                              [yes]
metatag                                       [answer]
who                                              child
text             Does {kid_name} have temper tantrums?
answer_type                                      scale
answer_tag                                         yes
metatags                                        answer
Name: 19756, dtype: object

In [240]:
if 'yes' in combined_answertags.at[19756, 'answer_tag']:
    print('yes')

yes


after the over-write

before the overwrite

In [58]:
muse_model_df[367].value_counts()

 1.0    1212
 0.0      69
 2.0      47
-1.0      37
-2.0       4
Name: 367, dtype: int64

after the overwrite - 367

In [71]:
muse_model_df[367].value_counts()

 1.0    1212
-2.0      92
 2.0      46
 0.0      46
-1.0      29
Name: 367, dtype: int64

added verification to see which ones would be over-written

In [73]:
muse_model_df[367].value_counts()

 1.0    1212
-3.0     120
 2.0      46
 0.0      46
-1.0       1
Name: 367, dtype: int64

In [105]:
muse_model_df[367].value_counts()

 1.0    1212
-2.0      92
 2.0      46
 0.0      46
-1.0      29
Name: 367, dtype: int64

In [60]:
muse_model_df[367].value_counts()

 1.0    1212
 0.0      69
 2.0      47
-1.0      37
-2.0       4
Name: 367, dtype: int64

In [69]:
muse_model_df[367].value_counts().sum()

1369

In [68]:
muse_model_df[368].value_counts().sum()

1242

In [74]:
muse_model_df[371].value_counts().sum()

2075

In [67]:
muse_model_df[368].value_counts()

 1.0    993
 0.0    108
-1.0     72
 2.0     66
-2.0      3
Name: 368, dtype: int64

In [ ]:
Q 371: "Do you work outside of the home?" | 2,012
Q 367: "Does {kid_name} attend school outside of the home?" | 1,349
Q 368: "Do you have reliable childcare for {kid_name}?" | 1,216
Q 24: "Do you know the name of {kid_name}'s primary doctor?" | 350
Q 13: "Do you know your next door neighbor's first name?" | 341
Q 25: "Does your family have health care?" | 339
Q 18: "Does your spouse work multiple jobs?" | 335
Q 590: "Do you have a primary doctor?" | 325
Q 15: "Does your spouse work full-time?" | 318
Q 41: "Are you the first person in your family to graduate college?" | 314

In [62]:
muse_model_df[367].value_counts()

 1.0    1212
 0.0      69
 2.0      47
-1.0      37
-2.0       4
Name: 367, dtype: int64

In [66]:
muse_model_df[371].value_counts()

 1.0    1534
-1.0      73
 2.0      68
 0.0      38
-2.0      11
Name: 371, dtype: int64

In [65]:
muse_model_df

,1,2,3,4,5,6,7,8,9,10,...,1154,1155,1156,1157,1158,1159,1160,1161,1162,1163
learner,,,,,,,,,,,,,,,,,,,,,
6.051420e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.134424e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.798113e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.756901e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.630121e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.628774e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.637258e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.722679e+15,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6.125581e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
answer_counts = muse_model_df.count()
answer_counts = list(answer_counts)

In [90]:
answer_counts.sum()

30854

In [88]:
for i in range(len(answer_counts)):
    if answer_counts[i] == 0:
        print(i)
        count += count
        
        

KeyError: 0

In [148]:
print(muse_model_df.count().sum())

30854


In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd

In [ ]:
X = combined_df.iloc[:, 1].values

In [ ]:
le = LabelEncoder()

In [ ]:
X = le.fit_transform(X)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [ ]:
categorical_features = categories
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
X = preprocessor.fit_transform(X)

In [ ]:
ohe = OneHotEncoder(categorical_features = categories)
X = ohe.fit_transform(X).toarray()

In [159]:
project = 'socos-quickstep-601'
dataset_id = 'muse_matrix'
table_id = 'learners_questions'
bucket_name = 'muse-matrix-data'

# Setup Storage vars
storage_client = storage.Client(project=project)
bucket = storage_client.get_bucket(bucket_name)

In [160]:
#destination
destination_uri = "gs://{}/{}".format(bucket_name, "muse-matrix.json")

In [161]:
#set job configuration to extract job as JSON files
job_config = bigquery.ExtractJobConfig()
job_config.destination_format = 'NEWLINE_DELIMITED_JSON'
job_config.write_disposition = 'WRITE_TRUNCATE'

In [162]:
# Setup BigQuery vars
bq_client = bigquery.Client()
dataset_ref = bq_client.dataset(dataset_id, project=project)
table_ref = dataset_ref.table(table_id)

In [163]:
extract_job = client.extract_table(
    table_ref,
    destination_uri,
    # Location must match that of the source table.
    location="US",
    job_config=job_config
)  # API request
extract_job.result()  # Waits for job to complete.

print(
    "Exported {}:{}.{} to {}".format(project, dataset_id, table_id, destination_uri)
)

BadRequest: 400 Table socos-quickstep-601:muse_matrix.learners_questions cannot be exported because it doesn't have a schema, or the schema is empty.

In [152]:
#project = 'smart-science'


In [164]:
#autodetect
dataset_ref = client.dataset(dataset_id)
job_config = bigquery.LoadJobConfig()
job_config.autodetect = True
job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
uri = destination_uri
load_job = client.load_table_from_uri(
    uri, dataset_ref.table("us_states"), job_config=job_config
)  # API request
print("Starting job {}".format(load_job.job_id))

load_job.result()  # Waits for table load to complete.
print("Job finished.")

destination_table = client.get_table(dataset_ref.table("us_states"))
print("Loaded {} rows.".format(destination_table.num_rows))

Starting job e368fb7f-e300-405f-b905-15709ce7fd23


NotFound: 404 Not found: URI gs://muse-matrix-data/muse-matrix.json

In [166]:
muse_model_df.to_gbq('muse_matrix.learners_questions', 
                 'socos-quickstep-601',
                 chunksize=None, 
                 if_exists='append',
                 verbose=False
                 )

/Users/dhyanna/socos/impact-index/env/lib/python3.7/site-packages/pandas_gbq/gbq.py:1127: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  stacklevel=1,


InvalidSchema: Please verify that the structure and data types in the DataFrame match the schema of the destination table.

In [176]:
muse_model_df.index = muse_model_df.index.astype(int)

In [178]:
muse_model_df.to_csv('muse-matrix.csv')

In [188]:
job_config.schema = [
    bigquery.SchemaField("1", "INTEGER"),
    bigquery.SchemaField("2", "STRING"),
]

dataset_id = 'muse_matrix'

dataset_ref = client.dataset(dataset_id)
job_config = bigquery.LoadJobConfig()
job_config.autodetect = True
job_config.source_format = 'CSV'
uri = 'gs://muse-matrix-data/muse-matrix.csv'
load_job = client.load_table_from_uri(
    uri, dataset_ref.table("us_states"), job_config=job_config
)  # API request
print("Starting job {}".format(load_job.job_id))

load_job.result()  # Waits for table load to complete.
print("Job finished.")

destination_table = client.get_table(dataset_ref.table("us_states"))
print("Loaded {} rows.".format(destination_table.num_rows))

Starting job 81e5d405-b55c-46d8-8ddf-3c607d09ceaf


BadRequest: 400 Error while reading data, error message: CSV table encountered too many errors, giving up. Rows: 607; errors: 1. Please look into the errors[] collection for more details.

In [ ]:
muse_model_df.iloc[607,:]

In [183]:
muse_model_df = muse_model_df.where(pd.notnull(muse_model_df), None)

In [186]:
muse_model_df.to_json('muse-matrix.json')

In [187]:
job_config.schema = [
    bigquery.SchemaField("name", "STRING"),
    bigquery.SchemaField("post_abbr", "STRING"),
]

In [190]:
muse_model_df.to_csv(
            'gs://muse-matrix-data/muse-matrix.csv',
            index=False
        )

In [195]:
new_cols = list(muse_model_df.columns)

for i in range(len(new_cols)):
    word = str(new_cols[i])
    new_cols[i] = 'q' + word

In [197]:
muse_model_df.columns = new_cols

In [198]:
muse_model_df

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,...,q1154,q1155,q1156,q1157,q1158,q1159,q1160,q1161,q1162,q1163
learner,,,,,,,,,,,,,,,,,,,,,
6051420109275136,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5134423787831296,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5798113272922112,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5756901417549824,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5630121163620352,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5628773986729984,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5637257553772544,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5722679185244160,None,None,None,None,None,None,1,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6125581443792896,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [225]:
project = 'socos-quickstep-601'
dataset_id = 'muse_matrix'
table_id = 'muse_matrix.learners_questions'

# Load job_config and define schema
job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("q1", "INTEGER"),
            bigquery.SchemaField("q2", "INTEGER"),
            bigquery.SchemaField("q3", "INTEGER"),
            bigquery.SchemaField("q4", "INTEGER"),
            bigquery.SchemaField("q5", "INTEGER"),
            bigquery.SchemaField("q6", "INTEGER"),
            bigquery.SchemaField("q7", "INTEGER"),
            bigquery.SchemaField("q8", "INTEGER"),
            bigquery.SchemaField("q9", "INTEGER"),
            bigquery.SchemaField("q10", "INTEGER"),
            bigquery.SchemaField("q11", "INTEGER"),
            bigquery.SchemaField("q12", "INTEGER"),
            bigquery.SchemaField("q13", "INTEGER"),
            bigquery.SchemaField("q14", "INTEGER"),
            bigquery.SchemaField("q15", "INTEGER"),
            bigquery.SchemaField("q16", "INTEGER"),
            bigquery.SchemaField("q17", "INTEGER"),
            bigquery.SchemaField("q18", "INTEGER"),
            bigquery.SchemaField("q19", "INTEGER"),
            bigquery.SchemaField("q20", "INTEGER"),
            bigquery.SchemaField("q21", "INTEGER"),
            bigquery.SchemaField("q22", "INTEGER"),
            bigquery.SchemaField("q23", "INTEGER"),
            bigquery.SchemaField("q24", "INTEGER"),
            bigquery.SchemaField("q25", "INTEGER"),
            bigquery.SchemaField("q26", "INTEGER"),
            bigquery.SchemaField("q27", "INTEGER"),
            bigquery.SchemaField("q28", "INTEGER"),
            bigquery.SchemaField("q29", "INTEGER"),
            bigquery.SchemaField("q30", "INTEGER"),
            bigquery.SchemaField("q31", "INTEGER"),
            bigquery.SchemaField("q32", "INTEGER"),
            bigquery.SchemaField("q33", "INTEGER"),
            bigquery.SchemaField("q34", "INTEGER"),
            bigquery.SchemaField("q35", "INTEGER"),
            bigquery.SchemaField("q36", "INTEGER"),
            bigquery.SchemaField("q37", "INTEGER"),
            bigquery.SchemaField("q38", "INTEGER"),
            bigquery.SchemaField("q39", "INTEGER"),
            bigquery.SchemaField("q40", "INTEGER"),
            bigquery.SchemaField("q41", "INTEGER"),
            bigquery.SchemaField("q42", "INTEGER"),
            bigquery.SchemaField("q43", "INTEGER"),
            bigquery.SchemaField("q44", "INTEGER"),
            bigquery.SchemaField("q45", "INTEGER"),
            bigquery.SchemaField("q46", "INTEGER"),
            bigquery.SchemaField("q47", "INTEGER"),
            bigquery.SchemaField("q48", "INTEGER"),
            bigquery.SchemaField("q49", "INTEGER"),
            bigquery.SchemaField("q50", "INTEGER"),
            bigquery.SchemaField("q51", "INTEGER"),
            bigquery.SchemaField("q52", "INTEGER"),
            bigquery.SchemaField("q53", "INTEGER"),
            bigquery.SchemaField("q54", "INTEGER"),
            bigquery.SchemaField("q55", "INTEGER"),
            bigquery.SchemaField("q56", "INTEGER"),
            bigquery.SchemaField("q57", "INTEGER"),
            bigquery.SchemaField("q58", "INTEGER"),
            bigquery.SchemaField("q59", "INTEGER"),
            bigquery.SchemaField("q60", "INTEGER"),
            bigquery.SchemaField("q61", "INTEGER"),
            bigquery.SchemaField("q62", "INTEGER"),
            bigquery.SchemaField("q63", "INTEGER"),
            bigquery.SchemaField("q64", "INTEGER"),
            bigquery.SchemaField("q65", "INTEGER"),
            bigquery.SchemaField("q66", "INTEGER"),
            bigquery.SchemaField("q67", "INTEGER"),
            bigquery.SchemaField("q68", "INTEGER"),
            bigquery.SchemaField("q69", "INTEGER"),
            bigquery.SchemaField("q70", "INTEGER"),
            bigquery.SchemaField("q71", "INTEGER"),
            bigquery.SchemaField("q72", "INTEGER"),
            bigquery.SchemaField("q73", "INTEGER"),
            bigquery.SchemaField("q74", "INTEGER"),
            bigquery.SchemaField("q75", "INTEGER"),
            bigquery.SchemaField("q76", "INTEGER"),
            bigquery.SchemaField("q77", "INTEGER"),
            bigquery.SchemaField("q78", "INTEGER"),
            bigquery.SchemaField("q79", "INTEGER"),
            bigquery.SchemaField("q80", "INTEGER"),
            bigquery.SchemaField("q81", "INTEGER"),
            bigquery.SchemaField("q82", "INTEGER"),
            bigquery.SchemaField("q83", "INTEGER"),
            bigquery.SchemaField("q84", "INTEGER"),
            bigquery.SchemaField("q85", "INTEGER"),
            bigquery.SchemaField("q86", "INTEGER"),
            bigquery.SchemaField("q87", "INTEGER"),
            bigquery.SchemaField("q88", "INTEGER"),
            bigquery.SchemaField("q89", "INTEGER"),
            bigquery.SchemaField("q90", "INTEGER"),
            bigquery.SchemaField("q91", "INTEGER"),
            bigquery.SchemaField("q92", "INTEGER"),
            bigquery.SchemaField("q93", "INTEGER"),
            bigquery.SchemaField("q94", "INTEGER"),
            bigquery.SchemaField("q95", "INTEGER"),
            bigquery.SchemaField("q96", "INTEGER"),
            bigquery.SchemaField("q97", "INTEGER"),
            bigquery.SchemaField("q98", "INTEGER"),
            bigquery.SchemaField("q99", "INTEGER"),
            bigquery.SchemaField("q100", "INTEGER"),
            bigquery.SchemaField("q101", "INTEGER"),
            bigquery.SchemaField("q102", "INTEGER"),
            bigquery.SchemaField("q103", "INTEGER"),
            bigquery.SchemaField("q104", "INTEGER"),
            bigquery.SchemaField("q105", "INTEGER"),
            bigquery.SchemaField("q106", "INTEGER"),
            bigquery.SchemaField("q107", "INTEGER"),
            bigquery.SchemaField("q108", "INTEGER"),
            bigquery.SchemaField("q109", "INTEGER"),
            bigquery.SchemaField("q110", "INTEGER"),
            bigquery.SchemaField("q111", "INTEGER"),
            bigquery.SchemaField("q112", "INTEGER"),
            bigquery.SchemaField("q113", "INTEGER"),
            bigquery.SchemaField("q114", "INTEGER"),
            bigquery.SchemaField("q115", "INTEGER"),
            bigquery.SchemaField("q116", "INTEGER"),
            bigquery.SchemaField("q117", "INTEGER"),
            bigquery.SchemaField("q118", "INTEGER"),
            bigquery.SchemaField("q119", "INTEGER"),
            bigquery.SchemaField("q120", "INTEGER"),
            bigquery.SchemaField("q121", "INTEGER"),
            bigquery.SchemaField("q122", "INTEGER"),
            bigquery.SchemaField("q123", "INTEGER"),
            bigquery.SchemaField("q124", "INTEGER"),
            bigquery.SchemaField("q125", "INTEGER"),
            bigquery.SchemaField("q126", "INTEGER"),
            bigquery.SchemaField("q127", "INTEGER"),
            bigquery.SchemaField("q128", "INTEGER"),
            bigquery.SchemaField("q129", "INTEGER"),
            bigquery.SchemaField("q130", "INTEGER"),
            bigquery.SchemaField("q131", "INTEGER"),
            bigquery.SchemaField("q132", "INTEGER"),
            bigquery.SchemaField("q133", "INTEGER"),
            bigquery.SchemaField("q134", "INTEGER"),
            bigquery.SchemaField("q135", "INTEGER"),
            bigquery.SchemaField("q136", "INTEGER"),
            bigquery.SchemaField("q137", "INTEGER"),
            bigquery.SchemaField("q138", "INTEGER"),
            bigquery.SchemaField("q139", "INTEGER"),
            bigquery.SchemaField("q140", "INTEGER"),
            bigquery.SchemaField("q141", "INTEGER"),
            bigquery.SchemaField("q142", "INTEGER"),
            bigquery.SchemaField("q143", "INTEGER"),
            bigquery.SchemaField("q144", "INTEGER"),
            bigquery.SchemaField("q145", "INTEGER"),
            bigquery.SchemaField("q146", "INTEGER"),
            bigquery.SchemaField("q147", "INTEGER"),
            bigquery.SchemaField("q148", "INTEGER"),
            bigquery.SchemaField("q149", "INTEGER"),
            bigquery.SchemaField("q150", "INTEGER"),
            bigquery.SchemaField("q151", "INTEGER"),
            bigquery.SchemaField("q152", "INTEGER"),
            bigquery.SchemaField("q153", "INTEGER"),
            bigquery.SchemaField("q154", "INTEGER"),
            bigquery.SchemaField("q155", "INTEGER"),
            bigquery.SchemaField("q156", "INTEGER"),
            bigquery.SchemaField("q157", "INTEGER"),
            bigquery.SchemaField("q158", "INTEGER"),
            bigquery.SchemaField("q159", "INTEGER"),
            bigquery.SchemaField("q160", "INTEGER"),
            bigquery.SchemaField("q161", "INTEGER"),
            bigquery.SchemaField("q162", "INTEGER"),
            bigquery.SchemaField("q163", "INTEGER"),
            bigquery.SchemaField("q164", "INTEGER"),
            bigquery.SchemaField("q165", "INTEGER"),
            bigquery.SchemaField("q166", "INTEGER"),
            bigquery.SchemaField("q167", "INTEGER"),
            bigquery.SchemaField("q168", "INTEGER"),
            bigquery.SchemaField("q169", "INTEGER"),
            bigquery.SchemaField("q170", "INTEGER"),
            bigquery.SchemaField("q171", "INTEGER"),
            bigquery.SchemaField("q172", "INTEGER"),
            bigquery.SchemaField("q173", "INTEGER"),
            bigquery.SchemaField("q174", "INTEGER"),
            bigquery.SchemaField("q175", "INTEGER"),
            bigquery.SchemaField("q176", "INTEGER"),
            bigquery.SchemaField("q177", "INTEGER"),
            bigquery.SchemaField("q178", "INTEGER"),
            bigquery.SchemaField("q179", "INTEGER"),
            bigquery.SchemaField("q180", "INTEGER"),
            bigquery.SchemaField("q181", "INTEGER"),
            bigquery.SchemaField("q182", "INTEGER"),
            bigquery.SchemaField("q183", "INTEGER"),
            bigquery.SchemaField("q184", "INTEGER"),
            bigquery.SchemaField("q185", "INTEGER"),
            bigquery.SchemaField("q186", "INTEGER"),
            bigquery.SchemaField("q187", "INTEGER"),
            bigquery.SchemaField("q188", "INTEGER"),
            bigquery.SchemaField("q189", "INTEGER"),
            bigquery.SchemaField("q190", "INTEGER"),
            bigquery.SchemaField("q191", "INTEGER"),
            bigquery.SchemaField("q192", "INTEGER"),
            bigquery.SchemaField("q193", "INTEGER"),
            bigquery.SchemaField("q194", "INTEGER"),
            bigquery.SchemaField("q195", "INTEGER"),
            bigquery.SchemaField("q196", "INTEGER"),
            bigquery.SchemaField("q197", "INTEGER"),
            bigquery.SchemaField("q198", "INTEGER"),
            bigquery.SchemaField("q199", "INTEGER"),
            bigquery.SchemaField("q200", "INTEGER"),
            bigquery.SchemaField("q201", "INTEGER"),
            bigquery.SchemaField("q202", "INTEGER"),
            bigquery.SchemaField("q203", "INTEGER"),
            bigquery.SchemaField("q204", "INTEGER"),
            bigquery.SchemaField("q205", "INTEGER"),
            bigquery.SchemaField("q206", "INTEGER"),
            bigquery.SchemaField("q207", "INTEGER"),
            bigquery.SchemaField("q208", "INTEGER"),
            bigquery.SchemaField("q209", "INTEGER"),
            bigquery.SchemaField("q210", "INTEGER"),
            bigquery.SchemaField("q211", "INTEGER"),
            bigquery.SchemaField("q212", "INTEGER"),
            bigquery.SchemaField("q213", "INTEGER"),
            bigquery.SchemaField("q214", "INTEGER"),
            bigquery.SchemaField("q215", "INTEGER"),
            bigquery.SchemaField("q216", "INTEGER"),
            bigquery.SchemaField("q217", "INTEGER"),
            bigquery.SchemaField("q218", "INTEGER"),
            bigquery.SchemaField("q219", "INTEGER"),
            bigquery.SchemaField("q220", "INTEGER"),
            bigquery.SchemaField("q221", "INTEGER"),
            bigquery.SchemaField("q222", "INTEGER"),
            bigquery.SchemaField("q223", "INTEGER"),
            bigquery.SchemaField("q224", "INTEGER"),
            bigquery.SchemaField("q225", "INTEGER"),
            bigquery.SchemaField("q226", "INTEGER"),
            bigquery.SchemaField("q227", "INTEGER"),
            bigquery.SchemaField("q228", "INTEGER"),
            bigquery.SchemaField("q229", "INTEGER"),
            bigquery.SchemaField("q230", "INTEGER"),
            bigquery.SchemaField("q231", "INTEGER"),
            bigquery.SchemaField("q232", "INTEGER"),
            bigquery.SchemaField("q233", "INTEGER"),
            bigquery.SchemaField("q234", "INTEGER"),
            bigquery.SchemaField("q235", "INTEGER"),
            bigquery.SchemaField("q236", "INTEGER"),
            bigquery.SchemaField("q237", "INTEGER"),
            bigquery.SchemaField("q238", "INTEGER"),
            bigquery.SchemaField("q239", "INTEGER"),
            bigquery.SchemaField("q240", "INTEGER"),
            bigquery.SchemaField("q241", "INTEGER"),
            bigquery.SchemaField("q242", "INTEGER"),
            bigquery.SchemaField("q243", "INTEGER"),
            bigquery.SchemaField("q244", "INTEGER"),
            bigquery.SchemaField("q245", "INTEGER"),
            bigquery.SchemaField("q246", "INTEGER"),
            bigquery.SchemaField("q247", "INTEGER"),
            bigquery.SchemaField("q248", "INTEGER"),
            bigquery.SchemaField("q249", "INTEGER"),
            bigquery.SchemaField("q250", "INTEGER"),
            bigquery.SchemaField("q251", "INTEGER"),
            bigquery.SchemaField("q252", "INTEGER"),
            bigquery.SchemaField("q253", "INTEGER"),
            bigquery.SchemaField("q254", "INTEGER"),
            bigquery.SchemaField("q255", "INTEGER"),
            bigquery.SchemaField("q256", "INTEGER"),
            bigquery.SchemaField("q257", "INTEGER"),
            bigquery.SchemaField("q258", "INTEGER"),
            bigquery.SchemaField("q259", "INTEGER"),
            bigquery.SchemaField("q260", "INTEGER"),
            bigquery.SchemaField("q261", "INTEGER"),
            bigquery.SchemaField("q262", "INTEGER"),
            bigquery.SchemaField("q263", "INTEGER"),
            bigquery.SchemaField("q264", "INTEGER"),
            bigquery.SchemaField("q265", "INTEGER"),
            bigquery.SchemaField("q266", "INTEGER"),
            bigquery.SchemaField("q267", "INTEGER"),
            bigquery.SchemaField("q268", "INTEGER"),
            bigquery.SchemaField("q269", "INTEGER"),
            bigquery.SchemaField("q270", "INTEGER"),
            bigquery.SchemaField("q271", "INTEGER"),
            bigquery.SchemaField("q272", "INTEGER"),
            bigquery.SchemaField("q273", "INTEGER"),
            bigquery.SchemaField("q274", "INTEGER"),
            bigquery.SchemaField("q275", "INTEGER"),
            bigquery.SchemaField("q276", "INTEGER"),
            bigquery.SchemaField("q277", "INTEGER"),
            bigquery.SchemaField("q278", "INTEGER"),
            bigquery.SchemaField("q279", "INTEGER"),
            bigquery.SchemaField("q280", "INTEGER"),
            bigquery.SchemaField("q281", "INTEGER"),
            bigquery.SchemaField("q282", "INTEGER"),
            bigquery.SchemaField("q283", "INTEGER"),
            bigquery.SchemaField("q284", "INTEGER"),
            bigquery.SchemaField("q285", "INTEGER"),
            bigquery.SchemaField("q286", "INTEGER"),
            bigquery.SchemaField("q287", "INTEGER"),
            bigquery.SchemaField("q288", "INTEGER"),
            bigquery.SchemaField("q289", "INTEGER"),
            bigquery.SchemaField("q290", "INTEGER"),
            bigquery.SchemaField("q291", "INTEGER"),
            bigquery.SchemaField("q292", "INTEGER"),
            bigquery.SchemaField("q293", "INTEGER"),
            bigquery.SchemaField("q294", "INTEGER"),
            bigquery.SchemaField("q295", "INTEGER"),
            bigquery.SchemaField("q296", "INTEGER"),
            bigquery.SchemaField("q297", "INTEGER"),
            bigquery.SchemaField("q298", "INTEGER"),
            bigquery.SchemaField("q299", "INTEGER"),
            bigquery.SchemaField("q300", "INTEGER"),
            bigquery.SchemaField("q301", "INTEGER"),
            bigquery.SchemaField("q302", "INTEGER"),
            bigquery.SchemaField("q303", "INTEGER"),
            bigquery.SchemaField("q304", "INTEGER"),
            bigquery.SchemaField("q305", "INTEGER"),
            bigquery.SchemaField("q306", "INTEGER"),
            bigquery.SchemaField("q307", "INTEGER"),
            bigquery.SchemaField("q308", "INTEGER"),
            bigquery.SchemaField("q309", "INTEGER"),
            bigquery.SchemaField("q310", "INTEGER"),
            bigquery.SchemaField("q311", "INTEGER"),
            bigquery.SchemaField("q312", "INTEGER"),
            bigquery.SchemaField("q313", "INTEGER"),
            bigquery.SchemaField("q314", "INTEGER"),
            bigquery.SchemaField("q315", "INTEGER"),
            bigquery.SchemaField("q316", "INTEGER"),
            bigquery.SchemaField("q317", "INTEGER"),
            bigquery.SchemaField("q318", "INTEGER"),
            bigquery.SchemaField("q319", "INTEGER"),
            bigquery.SchemaField("q320", "INTEGER"),
            bigquery.SchemaField("q321", "INTEGER"),
            bigquery.SchemaField("q322", "INTEGER"),
            bigquery.SchemaField("q323", "INTEGER"),
            bigquery.SchemaField("q324", "INTEGER"),
            bigquery.SchemaField("q325", "INTEGER"),
            bigquery.SchemaField("q326", "INTEGER"),
            bigquery.SchemaField("q327", "INTEGER"),
            bigquery.SchemaField("q328", "INTEGER"),
            bigquery.SchemaField("q329", "INTEGER"),
            bigquery.SchemaField("q330", "INTEGER"),
            bigquery.SchemaField("q331", "INTEGER"),
            bigquery.SchemaField("q332", "INTEGER"),
            bigquery.SchemaField("q333", "INTEGER"),
            bigquery.SchemaField("q334", "INTEGER"),
            bigquery.SchemaField("q335", "INTEGER"),
            bigquery.SchemaField("q336", "INTEGER"),
            bigquery.SchemaField("q337", "INTEGER"),
            bigquery.SchemaField("q338", "INTEGER"),
            bigquery.SchemaField("q339", "INTEGER"),
            bigquery.SchemaField("q340", "INTEGER"),
            bigquery.SchemaField("q341", "INTEGER"),
            bigquery.SchemaField("q342", "INTEGER"),
            bigquery.SchemaField("q343", "INTEGER"),
            bigquery.SchemaField("q344", "INTEGER"),
            bigquery.SchemaField("q345", "INTEGER"),
            bigquery.SchemaField("q346", "INTEGER"),
            bigquery.SchemaField("q347", "INTEGER"),
            bigquery.SchemaField("q348", "INTEGER"),
            bigquery.SchemaField("q349", "INTEGER"),
            bigquery.SchemaField("q350", "INTEGER"),
            bigquery.SchemaField("q351", "INTEGER"),
            bigquery.SchemaField("q352", "INTEGER"),
            bigquery.SchemaField("q353", "INTEGER"),
            bigquery.SchemaField("q354", "INTEGER"),
            bigquery.SchemaField("q355", "INTEGER"),
            bigquery.SchemaField("q356", "INTEGER"),
            bigquery.SchemaField("q357", "INTEGER"),
            bigquery.SchemaField("q358", "INTEGER"),
            bigquery.SchemaField("q359", "INTEGER"),
            bigquery.SchemaField("q360", "INTEGER"),
            bigquery.SchemaField("q361", "INTEGER"),
            bigquery.SchemaField("q362", "INTEGER"),
            bigquery.SchemaField("q363", "INTEGER"),
            bigquery.SchemaField("q364", "INTEGER"),
            bigquery.SchemaField("q365", "INTEGER"),
            bigquery.SchemaField("q366", "INTEGER"),
            bigquery.SchemaField("q367", "INTEGER"),
            bigquery.SchemaField("q368", "INTEGER"),
            bigquery.SchemaField("q369", "INTEGER"),
            bigquery.SchemaField("q370", "INTEGER"),
            bigquery.SchemaField("q371", "INTEGER"),
            bigquery.SchemaField("q372", "INTEGER"),
            bigquery.SchemaField("q373", "INTEGER"),
            bigquery.SchemaField("q374", "INTEGER"),
            bigquery.SchemaField("q375", "INTEGER"),
            bigquery.SchemaField("q376", "INTEGER"),
            bigquery.SchemaField("q377", "INTEGER"),
            bigquery.SchemaField("q378", "INTEGER"),
            bigquery.SchemaField("q379", "INTEGER"),
            bigquery.SchemaField("q380", "INTEGER"),
            bigquery.SchemaField("q381", "INTEGER"),
            bigquery.SchemaField("q382", "INTEGER"),
            bigquery.SchemaField("q383", "INTEGER"),
            bigquery.SchemaField("q384", "INTEGER"),
            bigquery.SchemaField("q385", "INTEGER"),
            bigquery.SchemaField("q386", "INTEGER"),
            bigquery.SchemaField("q387", "INTEGER"),
            bigquery.SchemaField("q388", "INTEGER"),
            bigquery.SchemaField("q389", "INTEGER"),
            bigquery.SchemaField("q390", "INTEGER"),
            bigquery.SchemaField("q391", "INTEGER"),
            bigquery.SchemaField("q392", "INTEGER"),
            bigquery.SchemaField("q393", "INTEGER"),
            bigquery.SchemaField("q394", "INTEGER"),
            bigquery.SchemaField("q395", "INTEGER"),
            bigquery.SchemaField("q396", "INTEGER"),
            bigquery.SchemaField("q397", "INTEGER"),
            bigquery.SchemaField("q398", "INTEGER"),
            bigquery.SchemaField("q399", "INTEGER"),
            bigquery.SchemaField("q400", "INTEGER"),
            bigquery.SchemaField("q401", "INTEGER"),
            bigquery.SchemaField("q402", "INTEGER"),
            bigquery.SchemaField("q403", "INTEGER"),
            bigquery.SchemaField("q404", "INTEGER"),
            bigquery.SchemaField("q405", "INTEGER"),
            bigquery.SchemaField("q406", "INTEGER"),
            bigquery.SchemaField("q407", "INTEGER"),
            bigquery.SchemaField("q408", "INTEGER"),
            bigquery.SchemaField("q409", "INTEGER"),
            bigquery.SchemaField("q410", "INTEGER"),
            bigquery.SchemaField("q411", "INTEGER"),
            bigquery.SchemaField("q412", "INTEGER"),
            bigquery.SchemaField("q413", "INTEGER"),
            bigquery.SchemaField("q414", "INTEGER"),
            bigquery.SchemaField("q415", "INTEGER"),
            bigquery.SchemaField("q416", "INTEGER"),
            bigquery.SchemaField("q417", "INTEGER"),
            bigquery.SchemaField("q418", "INTEGER"),
            bigquery.SchemaField("q419", "INTEGER"),
            bigquery.SchemaField("q420", "INTEGER"),
            bigquery.SchemaField("q421", "INTEGER"),
            bigquery.SchemaField("q422", "INTEGER"),
            bigquery.SchemaField("q423", "INTEGER"),
            bigquery.SchemaField("q424", "INTEGER"),
            bigquery.SchemaField("q425", "INTEGER"),
            bigquery.SchemaField("q426", "INTEGER"),
            bigquery.SchemaField("q427", "INTEGER"),
            bigquery.SchemaField("q428", "INTEGER"),
            bigquery.SchemaField("q429", "INTEGER"),
            bigquery.SchemaField("q430", "INTEGER"),
            bigquery.SchemaField("q431", "INTEGER"),
            bigquery.SchemaField("q432", "INTEGER"),
            bigquery.SchemaField("q433", "INTEGER"),
            bigquery.SchemaField("q434", "INTEGER"),
            bigquery.SchemaField("q435", "INTEGER"),
            bigquery.SchemaField("q436", "INTEGER"),
            bigquery.SchemaField("q437", "INTEGER"),
            bigquery.SchemaField("q438", "INTEGER"),
            bigquery.SchemaField("q439", "INTEGER"),
            bigquery.SchemaField("q440", "INTEGER"),
            bigquery.SchemaField("q441", "INTEGER"),
            bigquery.SchemaField("q442", "INTEGER"),
            bigquery.SchemaField("q443", "INTEGER"),
            bigquery.SchemaField("q444", "INTEGER"),
            bigquery.SchemaField("q445", "INTEGER"),
            bigquery.SchemaField("q446", "INTEGER"),
            bigquery.SchemaField("q447", "INTEGER"),
            bigquery.SchemaField("q448", "INTEGER"),
            bigquery.SchemaField("q449", "INTEGER"),
            bigquery.SchemaField("q450", "INTEGER"),
            bigquery.SchemaField("q451", "INTEGER"),
            bigquery.SchemaField("q452", "INTEGER"),
            bigquery.SchemaField("q453", "INTEGER"),
            bigquery.SchemaField("q454", "INTEGER"),
            bigquery.SchemaField("q455", "INTEGER"),
            bigquery.SchemaField("q456", "INTEGER"),
            bigquery.SchemaField("q457", "INTEGER"),
            bigquery.SchemaField("q458", "INTEGER"),
            bigquery.SchemaField("q459", "INTEGER"),
            bigquery.SchemaField("q460", "INTEGER"),
            bigquery.SchemaField("q461", "INTEGER"),
            bigquery.SchemaField("q462", "INTEGER"),
            bigquery.SchemaField("q463", "INTEGER"),
            bigquery.SchemaField("q464", "INTEGER"),
            bigquery.SchemaField("q465", "INTEGER"),
            bigquery.SchemaField("q466", "INTEGER"),
            bigquery.SchemaField("q467", "INTEGER"),
            bigquery.SchemaField("q468", "INTEGER"),
            bigquery.SchemaField("q469", "INTEGER"),
            bigquery.SchemaField("q470", "INTEGER"),
            bigquery.SchemaField("q471", "INTEGER"),
            bigquery.SchemaField("q472", "INTEGER"),
            bigquery.SchemaField("q473", "INTEGER"),
            bigquery.SchemaField("q474", "INTEGER"),
            bigquery.SchemaField("q475", "INTEGER"),
            bigquery.SchemaField("q476", "INTEGER"),
            bigquery.SchemaField("q477", "INTEGER"),
            bigquery.SchemaField("q478", "INTEGER"),
            bigquery.SchemaField("q479", "INTEGER"),
            bigquery.SchemaField("q480", "INTEGER"),
            bigquery.SchemaField("q481", "INTEGER"),
            bigquery.SchemaField("q482", "INTEGER"),
            bigquery.SchemaField("q483", "INTEGER"),
            bigquery.SchemaField("q484", "INTEGER"),
            bigquery.SchemaField("q485", "INTEGER"),
            bigquery.SchemaField("q486", "INTEGER"),
            bigquery.SchemaField("q487", "INTEGER"),
            bigquery.SchemaField("q488", "INTEGER"),
            bigquery.SchemaField("q489", "INTEGER"),
            bigquery.SchemaField("q490", "INTEGER"),
            bigquery.SchemaField("q491", "INTEGER"),
            bigquery.SchemaField("q492", "INTEGER"),
            bigquery.SchemaField("q493", "INTEGER"),
            bigquery.SchemaField("q494", "INTEGER"),
            bigquery.SchemaField("q495", "INTEGER"),
            bigquery.SchemaField("q496", "INTEGER"),
            bigquery.SchemaField("q497", "INTEGER"),
            bigquery.SchemaField("q498", "INTEGER"),
            bigquery.SchemaField("q499", "INTEGER"),
            bigquery.SchemaField("q500", "INTEGER"),
            bigquery.SchemaField("q501", "INTEGER"),
            bigquery.SchemaField("q502", "INTEGER"),
            bigquery.SchemaField("q503", "INTEGER"),
            bigquery.SchemaField("q504", "INTEGER"),
            bigquery.SchemaField("q505", "INTEGER"),
            bigquery.SchemaField("q506", "INTEGER"),
            bigquery.SchemaField("q507", "INTEGER"),
            bigquery.SchemaField("q508", "INTEGER"),
            bigquery.SchemaField("q509", "INTEGER"),
            bigquery.SchemaField("q510", "INTEGER"),
            bigquery.SchemaField("q511", "INTEGER"),
            bigquery.SchemaField("q512", "INTEGER"),
            bigquery.SchemaField("q513", "INTEGER"),
            bigquery.SchemaField("q514", "INTEGER"),
            bigquery.SchemaField("q515", "INTEGER"),
            bigquery.SchemaField("q516", "INTEGER"),
            bigquery.SchemaField("q517", "INTEGER"),
            bigquery.SchemaField("q518", "INTEGER"),
            bigquery.SchemaField("q519", "INTEGER"),
            bigquery.SchemaField("q520", "INTEGER"),
            bigquery.SchemaField("q521", "INTEGER"),
            bigquery.SchemaField("q522", "INTEGER"),
            bigquery.SchemaField("q523", "INTEGER"),
            bigquery.SchemaField("q524", "INTEGER"),
            bigquery.SchemaField("q525", "INTEGER"),
            bigquery.SchemaField("q526", "INTEGER"),
            bigquery.SchemaField("q527", "INTEGER"),
            bigquery.SchemaField("q528", "INTEGER"),
            bigquery.SchemaField("q529", "INTEGER"),
            bigquery.SchemaField("q530", "INTEGER"),
            bigquery.SchemaField("q531", "INTEGER"),
            bigquery.SchemaField("q532", "INTEGER"),
            bigquery.SchemaField("q533", "INTEGER"),
            bigquery.SchemaField("q534", "INTEGER"),
            bigquery.SchemaField("q535", "INTEGER"),
            bigquery.SchemaField("q536", "INTEGER"),
            bigquery.SchemaField("q537", "INTEGER"),
            bigquery.SchemaField("q538", "INTEGER"),
            bigquery.SchemaField("q539", "INTEGER"),
            bigquery.SchemaField("q540", "INTEGER"),
            bigquery.SchemaField("q541", "INTEGER"),
            bigquery.SchemaField("q542", "INTEGER"),
            bigquery.SchemaField("q543", "INTEGER"),
            bigquery.SchemaField("q544", "INTEGER"),
            bigquery.SchemaField("q545", "INTEGER"),
            bigquery.SchemaField("q546", "INTEGER"),
            bigquery.SchemaField("q547", "INTEGER"),
            bigquery.SchemaField("q548", "INTEGER"),
            bigquery.SchemaField("q549", "INTEGER"),
            bigquery.SchemaField("q550", "INTEGER"),
            bigquery.SchemaField("q551", "INTEGER"),
            bigquery.SchemaField("q552", "INTEGER"),
            bigquery.SchemaField("q553", "INTEGER"),
            bigquery.SchemaField("q554", "INTEGER"),
            bigquery.SchemaField("q555", "INTEGER"),
            bigquery.SchemaField("q556", "INTEGER"),
            bigquery.SchemaField("q557", "INTEGER"),
            bigquery.SchemaField("q558", "INTEGER"),
            bigquery.SchemaField("q559", "INTEGER"),
            bigquery.SchemaField("q560", "INTEGER"),
            bigquery.SchemaField("q561", "INTEGER"),
            bigquery.SchemaField("q562", "INTEGER"),
            bigquery.SchemaField("q563", "INTEGER"),
            bigquery.SchemaField("q564", "INTEGER"),
            bigquery.SchemaField("q565", "INTEGER"),
            bigquery.SchemaField("q566", "INTEGER"),
            bigquery.SchemaField("q567", "INTEGER"),
            bigquery.SchemaField("q568", "INTEGER"),
            bigquery.SchemaField("q569", "INTEGER"),
            bigquery.SchemaField("q570", "INTEGER"),
            bigquery.SchemaField("q571", "INTEGER"),
            bigquery.SchemaField("q572", "INTEGER"),
            bigquery.SchemaField("q573", "INTEGER"),
            bigquery.SchemaField("q574", "INTEGER"),
            bigquery.SchemaField("q575", "INTEGER"),
            bigquery.SchemaField("q576", "INTEGER"),
            bigquery.SchemaField("q577", "INTEGER"),
            bigquery.SchemaField("q578", "INTEGER"),
            bigquery.SchemaField("q579", "INTEGER"),
            bigquery.SchemaField("q580", "INTEGER"),
            bigquery.SchemaField("q581", "INTEGER"),
            bigquery.SchemaField("q582", "INTEGER"),
            bigquery.SchemaField("q583", "INTEGER"),
            bigquery.SchemaField("q584", "INTEGER"),
            bigquery.SchemaField("q585", "INTEGER"),
            bigquery.SchemaField("q586", "INTEGER"),
            bigquery.SchemaField("q587", "INTEGER"),
            bigquery.SchemaField("q588", "INTEGER"),
            bigquery.SchemaField("q589", "INTEGER"),
            bigquery.SchemaField("q590", "INTEGER"),
            bigquery.SchemaField("q591", "INTEGER"),
            bigquery.SchemaField("q592", "INTEGER"),
            bigquery.SchemaField("q593", "INTEGER"),
            bigquery.SchemaField("q594", "INTEGER"),
            bigquery.SchemaField("q595", "INTEGER"),
            bigquery.SchemaField("q596", "INTEGER"),
            bigquery.SchemaField("q597", "INTEGER"),
            bigquery.SchemaField("q598", "INTEGER"),
            bigquery.SchemaField("q599", "INTEGER"),
            bigquery.SchemaField("q600", "INTEGER"),
            bigquery.SchemaField("q601", "INTEGER"),
            bigquery.SchemaField("q602", "INTEGER"),
            bigquery.SchemaField("q603", "INTEGER"),
            bigquery.SchemaField("q604", "INTEGER"),
            bigquery.SchemaField("q605", "INTEGER"),
            bigquery.SchemaField("q606", "INTEGER"),
            bigquery.SchemaField("q607", "INTEGER"),
            bigquery.SchemaField("q608", "INTEGER"),
            bigquery.SchemaField("q609", "INTEGER"),
            bigquery.SchemaField("q610", "INTEGER"),
            bigquery.SchemaField("q611", "INTEGER"),
            bigquery.SchemaField("q612", "INTEGER"),
            bigquery.SchemaField("q613", "INTEGER"),
            bigquery.SchemaField("q614", "INTEGER"),
            bigquery.SchemaField("q615", "INTEGER"),
            bigquery.SchemaField("q616", "INTEGER"),
            bigquery.SchemaField("q617", "INTEGER"),
            bigquery.SchemaField("q618", "INTEGER"),
            bigquery.SchemaField("q619", "INTEGER"),
            bigquery.SchemaField("q620", "INTEGER"),
            bigquery.SchemaField("q621", "INTEGER"),
            bigquery.SchemaField("q622", "INTEGER"),
            bigquery.SchemaField("q623", "INTEGER"),
            bigquery.SchemaField("q624", "INTEGER"),
            bigquery.SchemaField("q625", "INTEGER"),
            bigquery.SchemaField("q626", "INTEGER"),
            bigquery.SchemaField("q627", "INTEGER"),
            bigquery.SchemaField("q628", "INTEGER"),
            bigquery.SchemaField("q629", "INTEGER"),
            bigquery.SchemaField("q630", "INTEGER"),
            bigquery.SchemaField("q631", "INTEGER"),
            bigquery.SchemaField("q632", "INTEGER"),
            bigquery.SchemaField("q633", "INTEGER"),
            bigquery.SchemaField("q634", "INTEGER"),
            bigquery.SchemaField("q635", "INTEGER"),
            bigquery.SchemaField("q636", "INTEGER"),
            bigquery.SchemaField("q637", "INTEGER"),
            bigquery.SchemaField("q638", "INTEGER"),
            bigquery.SchemaField("q639", "INTEGER"),
            bigquery.SchemaField("q640", "INTEGER"),
            bigquery.SchemaField("q641", "INTEGER"),
            bigquery.SchemaField("q642", "INTEGER"),
            bigquery.SchemaField("q643", "INTEGER"),
            bigquery.SchemaField("q644", "INTEGER"),
            bigquery.SchemaField("q645", "INTEGER"),
            bigquery.SchemaField("q646", "INTEGER"),
            bigquery.SchemaField("q647", "INTEGER"),
            bigquery.SchemaField("q648", "INTEGER"),
            bigquery.SchemaField("q649", "INTEGER"),
            bigquery.SchemaField("q650", "INTEGER"),
            bigquery.SchemaField("q651", "INTEGER"),
            bigquery.SchemaField("q652", "INTEGER"),
            bigquery.SchemaField("q653", "INTEGER"),
            bigquery.SchemaField("q654", "INTEGER"),
            bigquery.SchemaField("q655", "INTEGER"),
            bigquery.SchemaField("q656", "INTEGER"),
            bigquery.SchemaField("q657", "INTEGER"),
            bigquery.SchemaField("q658", "INTEGER"),
            bigquery.SchemaField("q659", "INTEGER"),
            bigquery.SchemaField("q660", "INTEGER"),
            bigquery.SchemaField("q661", "INTEGER"),
            bigquery.SchemaField("q662", "INTEGER"),
            bigquery.SchemaField("q663", "INTEGER"),
            bigquery.SchemaField("q664", "INTEGER"),
            bigquery.SchemaField("q665", "INTEGER"),
            bigquery.SchemaField("q666", "INTEGER"),
            bigquery.SchemaField("q667", "INTEGER"),
            bigquery.SchemaField("q668", "INTEGER"),
            bigquery.SchemaField("q669", "INTEGER"),
            bigquery.SchemaField("q670", "INTEGER"),
            bigquery.SchemaField("q671", "INTEGER"),
            bigquery.SchemaField("q672", "INTEGER"),
            bigquery.SchemaField("q673", "INTEGER"),
            bigquery.SchemaField("q674", "INTEGER"),
            bigquery.SchemaField("q675", "INTEGER"),
            bigquery.SchemaField("q676", "INTEGER"),
            bigquery.SchemaField("q677", "INTEGER"),
            bigquery.SchemaField("q678", "INTEGER"),
            bigquery.SchemaField("q679", "INTEGER"),
            bigquery.SchemaField("q680", "INTEGER"),
            bigquery.SchemaField("q681", "INTEGER"),
            bigquery.SchemaField("q682", "INTEGER"),
            bigquery.SchemaField("q683", "INTEGER"),
            bigquery.SchemaField("q684", "INTEGER"),
            bigquery.SchemaField("q685", "INTEGER"),
            bigquery.SchemaField("q686", "INTEGER"),
            bigquery.SchemaField("q687", "INTEGER"),
            bigquery.SchemaField("q688", "INTEGER"),
            bigquery.SchemaField("q689", "INTEGER"),
            bigquery.SchemaField("q690", "INTEGER"),
            bigquery.SchemaField("q691", "INTEGER"),
            bigquery.SchemaField("q692", "INTEGER"),
            bigquery.SchemaField("q693", "INTEGER"),
            bigquery.SchemaField("q694", "INTEGER"),
            bigquery.SchemaField("q695", "INTEGER"),
            bigquery.SchemaField("q696", "INTEGER"),
            bigquery.SchemaField("q697", "INTEGER"),
            bigquery.SchemaField("q698", "INTEGER"),
            bigquery.SchemaField("q699", "INTEGER"),
            bigquery.SchemaField("q700", "INTEGER"),
            bigquery.SchemaField("q701", "INTEGER"),
            bigquery.SchemaField("q702", "INTEGER"),
            bigquery.SchemaField("q703", "INTEGER"),
            bigquery.SchemaField("q704", "INTEGER"),
            bigquery.SchemaField("q705", "INTEGER"),
            bigquery.SchemaField("q706", "INTEGER"),
            bigquery.SchemaField("q707", "INTEGER"),
            bigquery.SchemaField("q708", "INTEGER"),
            bigquery.SchemaField("q709", "INTEGER"),
            bigquery.SchemaField("q710", "INTEGER"),
            bigquery.SchemaField("q711", "INTEGER"),
            bigquery.SchemaField("q712", "INTEGER"),
            bigquery.SchemaField("q713", "INTEGER"),
            bigquery.SchemaField("q714", "INTEGER"),
            bigquery.SchemaField("q715", "INTEGER"),
            bigquery.SchemaField("q716", "INTEGER"),
            bigquery.SchemaField("q717", "INTEGER"),
            bigquery.SchemaField("q718", "INTEGER"),
            bigquery.SchemaField("q719", "INTEGER"),
            bigquery.SchemaField("q720", "INTEGER"),
            bigquery.SchemaField("q721", "INTEGER"),
            bigquery.SchemaField("q722", "INTEGER"),
            bigquery.SchemaField("q723", "INTEGER"),
            bigquery.SchemaField("q724", "INTEGER"),
            bigquery.SchemaField("q725", "INTEGER"),
            bigquery.SchemaField("q726", "INTEGER"),
            bigquery.SchemaField("q727", "INTEGER"),
            bigquery.SchemaField("q728", "INTEGER"),
            bigquery.SchemaField("q729", "INTEGER"),
            bigquery.SchemaField("q730", "INTEGER"),
            bigquery.SchemaField("q731", "INTEGER"),
            bigquery.SchemaField("q732", "INTEGER"),
            bigquery.SchemaField("q733", "INTEGER"),
            bigquery.SchemaField("q734", "INTEGER"),
            bigquery.SchemaField("q735", "INTEGER"),
            bigquery.SchemaField("q736", "INTEGER"),
            bigquery.SchemaField("q737", "INTEGER"),
            bigquery.SchemaField("q738", "INTEGER"),
            bigquery.SchemaField("q739", "INTEGER"),
            bigquery.SchemaField("q740", "INTEGER"),
            bigquery.SchemaField("q741", "INTEGER"),
            bigquery.SchemaField("q742", "INTEGER"),
            bigquery.SchemaField("q743", "INTEGER"),
            bigquery.SchemaField("q744", "INTEGER"),
            bigquery.SchemaField("q745", "INTEGER"),
            bigquery.SchemaField("q746", "INTEGER"),
            bigquery.SchemaField("q747", "INTEGER"),
            bigquery.SchemaField("q748", "INTEGER"),
            bigquery.SchemaField("q749", "INTEGER"),
            bigquery.SchemaField("q750", "INTEGER"),
            bigquery.SchemaField("q751", "INTEGER"),
            bigquery.SchemaField("q752", "INTEGER"),
            bigquery.SchemaField("q753", "INTEGER"),
            bigquery.SchemaField("q754", "INTEGER"),
            bigquery.SchemaField("q755", "INTEGER"),
            bigquery.SchemaField("q756", "INTEGER"),
            bigquery.SchemaField("q757", "INTEGER"),
            bigquery.SchemaField("q758", "INTEGER"),
            bigquery.SchemaField("q759", "INTEGER"),
            bigquery.SchemaField("q760", "INTEGER"),
            bigquery.SchemaField("q761", "INTEGER"),
            bigquery.SchemaField("q762", "INTEGER"),
            bigquery.SchemaField("q763", "INTEGER"),
            bigquery.SchemaField("q764", "INTEGER"),
            bigquery.SchemaField("q765", "INTEGER"),
            bigquery.SchemaField("q766", "INTEGER"),
            bigquery.SchemaField("q767", "INTEGER"),
            bigquery.SchemaField("q768", "INTEGER"),
            bigquery.SchemaField("q769", "INTEGER"),
            bigquery.SchemaField("q770", "INTEGER"),
            bigquery.SchemaField("q771", "INTEGER"),
            bigquery.SchemaField("q772", "INTEGER"),
            bigquery.SchemaField("q773", "INTEGER"),
            bigquery.SchemaField("q774", "INTEGER"),
            bigquery.SchemaField("q775", "INTEGER"),
            bigquery.SchemaField("q776", "INTEGER"),
            bigquery.SchemaField("q777", "INTEGER"),
            bigquery.SchemaField("q778", "INTEGER"),
            bigquery.SchemaField("q779", "INTEGER"),
            bigquery.SchemaField("q780", "INTEGER"),
            bigquery.SchemaField("q781", "INTEGER"),
            bigquery.SchemaField("q782", "INTEGER"),
            bigquery.SchemaField("q783", "INTEGER"),
            bigquery.SchemaField("q784", "INTEGER"),
            bigquery.SchemaField("q785", "INTEGER"),
            bigquery.SchemaField("q786", "INTEGER"),
            bigquery.SchemaField("q787", "INTEGER"),
            bigquery.SchemaField("q788", "INTEGER"),
            bigquery.SchemaField("q789", "INTEGER"),
            bigquery.SchemaField("q790", "INTEGER"),
            bigquery.SchemaField("q791", "INTEGER"),
            bigquery.SchemaField("q792", "INTEGER"),
            bigquery.SchemaField("q793", "INTEGER"),
            bigquery.SchemaField("q794", "INTEGER"),
            bigquery.SchemaField("q795", "INTEGER"),
            bigquery.SchemaField("q796", "INTEGER"),
            bigquery.SchemaField("q797", "INTEGER"),
            bigquery.SchemaField("q798", "INTEGER"),
            bigquery.SchemaField("q799", "INTEGER"),
            bigquery.SchemaField("q800", "INTEGER"),
            bigquery.SchemaField("q801", "INTEGER"),
            bigquery.SchemaField("q802", "INTEGER"),
            bigquery.SchemaField("q803", "INTEGER"),
            bigquery.SchemaField("q804", "INTEGER"),
            bigquery.SchemaField("q805", "INTEGER"),
            bigquery.SchemaField("q806", "INTEGER"),
            bigquery.SchemaField("q807", "INTEGER"),
            bigquery.SchemaField("q808", "INTEGER"),
            bigquery.SchemaField("q809", "INTEGER"),
            bigquery.SchemaField("q810", "INTEGER"),
            bigquery.SchemaField("q811", "INTEGER"),
            bigquery.SchemaField("q812", "INTEGER"),
            bigquery.SchemaField("q813", "INTEGER"),
            bigquery.SchemaField("q814", "INTEGER"),
            bigquery.SchemaField("q815", "INTEGER"),
            bigquery.SchemaField("q816", "INTEGER"),
            bigquery.SchemaField("q817", "INTEGER"),
            bigquery.SchemaField("q818", "INTEGER"),
            bigquery.SchemaField("q819", "INTEGER"),
            bigquery.SchemaField("q820", "INTEGER"),
            bigquery.SchemaField("q821", "INTEGER"),
            bigquery.SchemaField("q822", "INTEGER"),
            bigquery.SchemaField("q823", "INTEGER"),
            bigquery.SchemaField("q824", "INTEGER"),
            bigquery.SchemaField("q825", "INTEGER"),
            bigquery.SchemaField("q826", "INTEGER"),
            bigquery.SchemaField("q827", "INTEGER"),
            bigquery.SchemaField("q828", "INTEGER"),
            bigquery.SchemaField("q829", "INTEGER"),
            bigquery.SchemaField("q830", "INTEGER"),
            bigquery.SchemaField("q831", "INTEGER"),
            bigquery.SchemaField("q832", "INTEGER"),
            bigquery.SchemaField("q833", "INTEGER"),
            bigquery.SchemaField("q834", "INTEGER"),
            bigquery.SchemaField("q835", "INTEGER"),
            bigquery.SchemaField("q836", "INTEGER"),
            bigquery.SchemaField("q837", "INTEGER"),
            bigquery.SchemaField("q838", "INTEGER"),
            bigquery.SchemaField("q839", "INTEGER"),
            bigquery.SchemaField("q840", "INTEGER"),
            bigquery.SchemaField("q841", "INTEGER"),
            bigquery.SchemaField("q842", "INTEGER"),
            bigquery.SchemaField("q843", "INTEGER"),
            bigquery.SchemaField("q844", "INTEGER"),
            bigquery.SchemaField("q845", "INTEGER"),
            bigquery.SchemaField("q846", "INTEGER"),
            bigquery.SchemaField("q847", "INTEGER"),
            bigquery.SchemaField("q848", "INTEGER"),
            bigquery.SchemaField("q849", "INTEGER"),
            bigquery.SchemaField("q850", "INTEGER"),
            bigquery.SchemaField("q851", "INTEGER"),
            bigquery.SchemaField("q852", "INTEGER"),
            bigquery.SchemaField("q853", "INTEGER"),
            bigquery.SchemaField("q854", "INTEGER"),
            bigquery.SchemaField("q855", "INTEGER"),
            bigquery.SchemaField("q856", "INTEGER"),
            bigquery.SchemaField("q857", "INTEGER"),
            bigquery.SchemaField("q858", "INTEGER"),
            bigquery.SchemaField("q859", "INTEGER"),
            bigquery.SchemaField("q860", "INTEGER"),
            bigquery.SchemaField("q861", "INTEGER"),
            bigquery.SchemaField("q862", "INTEGER"),
            bigquery.SchemaField("q863", "INTEGER"),
            bigquery.SchemaField("q864", "INTEGER"),
            bigquery.SchemaField("q865", "INTEGER"),
            bigquery.SchemaField("q866", "INTEGER"),
            bigquery.SchemaField("q867", "INTEGER"),
            bigquery.SchemaField("q868", "INTEGER"),
            bigquery.SchemaField("q869", "INTEGER"),
            bigquery.SchemaField("q870", "INTEGER"),
            bigquery.SchemaField("q871", "INTEGER"),
            bigquery.SchemaField("q872", "INTEGER"),
            bigquery.SchemaField("q873", "INTEGER"),
            bigquery.SchemaField("q874", "INTEGER"),
            bigquery.SchemaField("q875", "INTEGER"),
            bigquery.SchemaField("q876", "INTEGER"),
            bigquery.SchemaField("q877", "INTEGER"),
            bigquery.SchemaField("q878", "INTEGER"),
            bigquery.SchemaField("q879", "INTEGER"),
            bigquery.SchemaField("q880", "INTEGER"),
            bigquery.SchemaField("q881", "INTEGER"),
            bigquery.SchemaField("q882", "INTEGER"),
            bigquery.SchemaField("q883", "INTEGER"),
            bigquery.SchemaField("q884", "INTEGER"),
            bigquery.SchemaField("q885", "INTEGER"),
            bigquery.SchemaField("q886", "INTEGER"),
            bigquery.SchemaField("q887", "INTEGER"),
            bigquery.SchemaField("q888", "INTEGER"),
            bigquery.SchemaField("q889", "INTEGER"),
            bigquery.SchemaField("q890", "INTEGER"),
            bigquery.SchemaField("q891", "INTEGER"),
            bigquery.SchemaField("q892", "INTEGER"),
            bigquery.SchemaField("q893", "INTEGER"),
            bigquery.SchemaField("q894", "INTEGER"),
            bigquery.SchemaField("q895", "INTEGER"),
            bigquery.SchemaField("q896", "INTEGER"),
            bigquery.SchemaField("q897", "INTEGER"),
            bigquery.SchemaField("q898", "INTEGER"),
            bigquery.SchemaField("q899", "INTEGER"),
            bigquery.SchemaField("q900", "INTEGER"),
            bigquery.SchemaField("q901", "INTEGER"),
            bigquery.SchemaField("q902", "INTEGER"),
            bigquery.SchemaField("q903", "INTEGER"),
            bigquery.SchemaField("q904", "INTEGER"),
            bigquery.SchemaField("q905", "INTEGER"),
            bigquery.SchemaField("q906", "INTEGER"),
            bigquery.SchemaField("q907", "INTEGER"),
            bigquery.SchemaField("q908", "INTEGER"),
            bigquery.SchemaField("q909", "INTEGER"),
            bigquery.SchemaField("q910", "INTEGER"),
            bigquery.SchemaField("q911", "INTEGER"),
            bigquery.SchemaField("q912", "INTEGER"),
            bigquery.SchemaField("q913", "INTEGER"),
            bigquery.SchemaField("q914", "INTEGER"),
            bigquery.SchemaField("q915", "INTEGER"),
            bigquery.SchemaField("q916", "INTEGER"),
            bigquery.SchemaField("q917", "INTEGER"),
            bigquery.SchemaField("q918", "INTEGER"),
            bigquery.SchemaField("q919", "INTEGER"),
            bigquery.SchemaField("q920", "INTEGER"),
            bigquery.SchemaField("q921", "INTEGER"),
            bigquery.SchemaField("q922", "INTEGER"),
            bigquery.SchemaField("q923", "INTEGER"),
            bigquery.SchemaField("q924", "INTEGER"),
            bigquery.SchemaField("q925", "INTEGER"),
            bigquery.SchemaField("q926", "INTEGER"),
            bigquery.SchemaField("q927", "INTEGER"),
            bigquery.SchemaField("q928", "INTEGER"),
            bigquery.SchemaField("q929", "INTEGER"),
            bigquery.SchemaField("q930", "INTEGER"),
            bigquery.SchemaField("q931", "INTEGER"),
            bigquery.SchemaField("q932", "INTEGER"),
            bigquery.SchemaField("q933", "INTEGER"),
            bigquery.SchemaField("q934", "INTEGER"),
            bigquery.SchemaField("q935", "INTEGER"),
            bigquery.SchemaField("q936", "INTEGER"),
            bigquery.SchemaField("q937", "INTEGER"),
            bigquery.SchemaField("q938", "INTEGER"),
            bigquery.SchemaField("q939", "INTEGER"),
            bigquery.SchemaField("q940", "INTEGER"),
            bigquery.SchemaField("q941", "INTEGER"),
            bigquery.SchemaField("q942", "INTEGER"),
            bigquery.SchemaField("q943", "INTEGER"),
            bigquery.SchemaField("q944", "INTEGER"),
            bigquery.SchemaField("q945", "INTEGER"),
            bigquery.SchemaField("q946", "INTEGER"),
            bigquery.SchemaField("q947", "INTEGER"),
            bigquery.SchemaField("q948", "INTEGER"),
            bigquery.SchemaField("q949", "INTEGER"),
            bigquery.SchemaField("q950", "INTEGER"),
            bigquery.SchemaField("q951", "INTEGER"),
            bigquery.SchemaField("q952", "INTEGER"),
            bigquery.SchemaField("q953", "INTEGER"),
            bigquery.SchemaField("q954", "INTEGER"),
            bigquery.SchemaField("q955", "INTEGER"),
            bigquery.SchemaField("q956", "INTEGER"),
            bigquery.SchemaField("q957", "INTEGER"),
            bigquery.SchemaField("q958", "INTEGER"),
            bigquery.SchemaField("q959", "INTEGER"),
            bigquery.SchemaField("q960", "INTEGER"),
            bigquery.SchemaField("q961", "INTEGER"),
            bigquery.SchemaField("q962", "INTEGER"),
            bigquery.SchemaField("q963", "INTEGER"),
            bigquery.SchemaField("q964", "INTEGER"),
            bigquery.SchemaField("q965", "INTEGER"),
            bigquery.SchemaField("q966", "INTEGER"),
            bigquery.SchemaField("q967", "INTEGER"),
            bigquery.SchemaField("q968", "INTEGER"),
            bigquery.SchemaField("q969", "INTEGER"),
            bigquery.SchemaField("q970", "INTEGER"),
            bigquery.SchemaField("q971", "INTEGER"),
            bigquery.SchemaField("q972", "INTEGER"),
            bigquery.SchemaField("q973", "INTEGER"),
            bigquery.SchemaField("q974", "INTEGER"),
            bigquery.SchemaField("q975", "INTEGER"),
            bigquery.SchemaField("q976", "INTEGER"),
            bigquery.SchemaField("q977", "INTEGER"),
            bigquery.SchemaField("q978", "INTEGER"),
            bigquery.SchemaField("q979", "INTEGER"),
            bigquery.SchemaField("q980", "INTEGER"),
            bigquery.SchemaField("q981", "INTEGER"),
            bigquery.SchemaField("q982", "INTEGER"),
            bigquery.SchemaField("q983", "INTEGER"),
            bigquery.SchemaField("q984", "INTEGER"),
            bigquery.SchemaField("q985", "INTEGER"),
            bigquery.SchemaField("q986", "INTEGER"),
            bigquery.SchemaField("q987", "INTEGER"),
            bigquery.SchemaField("q988", "INTEGER"),
            bigquery.SchemaField("q989", "INTEGER"),
            bigquery.SchemaField("q990", "INTEGER"),
            bigquery.SchemaField("q991", "INTEGER"),
            bigquery.SchemaField("q992", "INTEGER"),
            bigquery.SchemaField("q993", "INTEGER"),
            bigquery.SchemaField("q994", "INTEGER"),
            bigquery.SchemaField("q995", "INTEGER"),
            bigquery.SchemaField("q996", "INTEGER"),
            bigquery.SchemaField("q997", "INTEGER"),
            bigquery.SchemaField("q998", "INTEGER"),
            bigquery.SchemaField("q999", "INTEGER"),
            bigquery.SchemaField("q1000", "INTEGER"),
            bigquery.SchemaField("q1001", "INTEGER"),
            bigquery.SchemaField("q1002", "INTEGER"),
            bigquery.SchemaField("q1003", "INTEGER"),
            bigquery.SchemaField("q1004", "INTEGER"),
            bigquery.SchemaField("q1005", "INTEGER"),
            bigquery.SchemaField("q1006", "INTEGER"),
            bigquery.SchemaField("q1007", "INTEGER"),
            bigquery.SchemaField("q1008", "INTEGER"),
            bigquery.SchemaField("q1009", "INTEGER"),
            bigquery.SchemaField("q1010", "INTEGER"),
            bigquery.SchemaField("q1011", "INTEGER"),
            bigquery.SchemaField("q1012", "INTEGER"),
            bigquery.SchemaField("q1013", "INTEGER"),
            bigquery.SchemaField("q1014", "INTEGER"),
            bigquery.SchemaField("q1015", "INTEGER"),
            bigquery.SchemaField("q1016", "INTEGER"),
            bigquery.SchemaField("q1017", "INTEGER"),
            bigquery.SchemaField("q1018", "INTEGER"),
            bigquery.SchemaField("q1019", "INTEGER"),
            bigquery.SchemaField("q1020", "INTEGER"),
            bigquery.SchemaField("q1021", "INTEGER"),
            bigquery.SchemaField("q1022", "INTEGER"),
            bigquery.SchemaField("q1023", "INTEGER"),
            bigquery.SchemaField("q1024", "INTEGER"),
            bigquery.SchemaField("q1025", "INTEGER"),
            bigquery.SchemaField("q1026", "INTEGER"),
            bigquery.SchemaField("q1027", "INTEGER"),
            bigquery.SchemaField("q1028", "INTEGER"),
            bigquery.SchemaField("q1029", "INTEGER"),
            bigquery.SchemaField("q1030", "INTEGER"),
            bigquery.SchemaField("q1031", "INTEGER"),
            bigquery.SchemaField("q1032", "INTEGER"),
            bigquery.SchemaField("q1033", "INTEGER"),
            bigquery.SchemaField("q1034", "INTEGER"),
            bigquery.SchemaField("q1035", "INTEGER"),
            bigquery.SchemaField("q1036", "INTEGER"),
            bigquery.SchemaField("q1037", "INTEGER"),
            bigquery.SchemaField("q1038", "INTEGER"),
            bigquery.SchemaField("q1039", "INTEGER"),
            bigquery.SchemaField("q1040", "INTEGER"),
            bigquery.SchemaField("q1041", "INTEGER"),
            bigquery.SchemaField("q1042", "INTEGER"),
            bigquery.SchemaField("q1043", "INTEGER"),
            bigquery.SchemaField("q1044", "INTEGER"),
            bigquery.SchemaField("q1045", "INTEGER"),
            bigquery.SchemaField("q1046", "INTEGER"),
            bigquery.SchemaField("q1047", "INTEGER"),
            bigquery.SchemaField("q1048", "INTEGER"),
            bigquery.SchemaField("q1049", "INTEGER"),
            bigquery.SchemaField("q1050", "INTEGER"),
            bigquery.SchemaField("q1051", "INTEGER"),
            bigquery.SchemaField("q1052", "INTEGER"),
            bigquery.SchemaField("q1053", "INTEGER"),
            bigquery.SchemaField("q1054", "INTEGER"),
            bigquery.SchemaField("q1055", "INTEGER"),
            bigquery.SchemaField("q1056", "INTEGER"),
            bigquery.SchemaField("q1057", "INTEGER"),
            bigquery.SchemaField("q1058", "INTEGER"),
            bigquery.SchemaField("q1059", "INTEGER"),
            bigquery.SchemaField("q1060", "INTEGER"),
            bigquery.SchemaField("q1061", "INTEGER"),
            bigquery.SchemaField("q1062", "INTEGER"),
            bigquery.SchemaField("q1063", "INTEGER"),
            bigquery.SchemaField("q1064", "INTEGER"),
            bigquery.SchemaField("q1065", "INTEGER"),
            bigquery.SchemaField("q1066", "INTEGER"),
            bigquery.SchemaField("q1067", "INTEGER"),
            bigquery.SchemaField("q1068", "INTEGER"),
            bigquery.SchemaField("q1069", "INTEGER"),
            bigquery.SchemaField("q1070", "INTEGER"),
            bigquery.SchemaField("q1071", "INTEGER"),
            bigquery.SchemaField("q1072", "INTEGER"),
            bigquery.SchemaField("q1073", "INTEGER"),
            bigquery.SchemaField("q1074", "INTEGER"),
            bigquery.SchemaField("q1075", "INTEGER"),
            bigquery.SchemaField("q1076", "INTEGER"),
            bigquery.SchemaField("q1077", "INTEGER"),
            bigquery.SchemaField("q1078", "INTEGER"),
            bigquery.SchemaField("q1079", "INTEGER"),
            bigquery.SchemaField("q1080", "INTEGER"),
            bigquery.SchemaField("q1081", "INTEGER"),
            bigquery.SchemaField("q1082", "INTEGER"),
            bigquery.SchemaField("q1083", "INTEGER"),
            bigquery.SchemaField("q1084", "INTEGER"),
            bigquery.SchemaField("q1085", "INTEGER"),
            bigquery.SchemaField("q1086", "INTEGER"),
            bigquery.SchemaField("q1087", "INTEGER"),
            bigquery.SchemaField("q1088", "INTEGER"),
            bigquery.SchemaField("q1089", "INTEGER"),
            bigquery.SchemaField("q1090", "INTEGER"),
            bigquery.SchemaField("q1091", "INTEGER"),
            bigquery.SchemaField("q1092", "INTEGER"),
            bigquery.SchemaField("q1093", "INTEGER"),
            bigquery.SchemaField("q1094", "INTEGER"),
            bigquery.SchemaField("q1095", "INTEGER"),
            bigquery.SchemaField("q1096", "INTEGER"),
            bigquery.SchemaField("q1097", "INTEGER"),
            bigquery.SchemaField("q1098", "INTEGER"),
            bigquery.SchemaField("q1099", "INTEGER"),
            bigquery.SchemaField("q1100", "INTEGER"),
            bigquery.SchemaField("q1101", "INTEGER"),
            bigquery.SchemaField("q1102", "INTEGER"),
            bigquery.SchemaField("q1103", "INTEGER"),
            bigquery.SchemaField("q1104", "INTEGER"),
            bigquery.SchemaField("q1105", "INTEGER"),
            bigquery.SchemaField("q1106", "INTEGER"),
            #does not exist
            #bigquery.SchemaField("q1107", "INTEGER"),
            bigquery.SchemaField("q1108", "INTEGER"),
            bigquery.SchemaField("q1109", "INTEGER"),
            bigquery.SchemaField("q1110", "INTEGER"),
            bigquery.SchemaField("q1111", "INTEGER"),
            bigquery.SchemaField("q1112", "INTEGER"),
            bigquery.SchemaField("q1113", "INTEGER"),
            bigquery.SchemaField("q1114", "INTEGER"),
            bigquery.SchemaField("q1115", "INTEGER"),
            bigquery.SchemaField("q1116", "INTEGER"),
            bigquery.SchemaField("q1117", "INTEGER"),
            bigquery.SchemaField("q1118", "INTEGER"),
            bigquery.SchemaField("q1119", "INTEGER"),
            bigquery.SchemaField("q1120", "INTEGER"),
            bigquery.SchemaField("q1121", "INTEGER"),
            bigquery.SchemaField("q1122", "INTEGER"),
            bigquery.SchemaField("q1123", "INTEGER"),
            bigquery.SchemaField("q1124", "INTEGER"),
            bigquery.SchemaField("q1125", "INTEGER"),
            bigquery.SchemaField("q1126", "INTEGER"),
            bigquery.SchemaField("q1127", "INTEGER"),
            bigquery.SchemaField("q1128", "INTEGER"),
            bigquery.SchemaField("q1129", "INTEGER"),
            bigquery.SchemaField("q1130", "INTEGER"),
            bigquery.SchemaField("q1131", "INTEGER"),
            bigquery.SchemaField("q1132", "INTEGER"),
            bigquery.SchemaField("q1133", "INTEGER"),
            bigquery.SchemaField("q1134", "INTEGER"),
            bigquery.SchemaField("q1135", "INTEGER"),
            bigquery.SchemaField("q1136", "INTEGER"),
            bigquery.SchemaField("q1137", "INTEGER"),
            bigquery.SchemaField("q1138", "INTEGER"),
            bigquery.SchemaField("q1139", "INTEGER"),
            bigquery.SchemaField("q1140", "INTEGER"),
            bigquery.SchemaField("q1141", "INTEGER"),
            bigquery.SchemaField("q1142", "INTEGER"),
            bigquery.SchemaField("q1143", "INTEGER"),
            bigquery.SchemaField("q1144", "INTEGER"),
            bigquery.SchemaField("q1145", "INTEGER"),
            bigquery.SchemaField("q1146", "INTEGER"),
            bigquery.SchemaField("q1147", "INTEGER"),
            bigquery.SchemaField("q1148", "INTEGER"),
            bigquery.SchemaField("q1149", "INTEGER"),
            bigquery.SchemaField("q1150", "INTEGER"),
            bigquery.SchemaField("q1151", "INTEGER"),
            bigquery.SchemaField("q1152", "INTEGER"),
            bigquery.SchemaField("q1153", "INTEGER"),
            bigquery.SchemaField("q1154", "INTEGER"),
            bigquery.SchemaField("q1155", "INTEGER"),
            bigquery.SchemaField("q1156", "INTEGER"),
            bigquery.SchemaField("q1157", "INTEGER"),
            bigquery.SchemaField("q1158", "INTEGER"),
            bigquery.SchemaField("q1159", "INTEGER"),
            bigquery.SchemaField("q1160", "INTEGER"),
            bigquery.SchemaField("q1161", "INTEGER"),
            bigquery.SchemaField("q1162", "INTEGER"),
            bigquery.SchemaField("q1163", "INTEGER")
        ],
        # Set job configuration write_disposition to be able to overwrite existing table
    write_disposition='WRITE_TRUNCATE'
    )

    # Define job
job = client.load_table_from_dataframe(
    muse_model_df,
    table_id,
    job_config=job_config,
    location='US'
)

    # Wait for the load job to complete.
job.result()
    
    # Make an API request
table = client.get_table(table_id)
print(
    'Loaded {} rows with {} columns to {}'.format(table.num_rows, len(table.schema), table_id)
    )

Loaded 2967 rows with 1162 columns to muse_matrix.learners_questions


In [212]:
#bigquery.SchemaField("1", "STRING"),
for i in range(1163):
    print (f"bigquery.SchemaField(\"q{i}\", \"INTEGER\"),")

bigquery.SchemaField("q0", "INTEGER"),
bigquery.SchemaField("q1", "INTEGER"),
bigquery.SchemaField("q2", "INTEGER"),
bigquery.SchemaField("q3", "INTEGER"),
bigquery.SchemaField("q4", "INTEGER"),
bigquery.SchemaField("q5", "INTEGER"),
bigquery.SchemaField("q6", "INTEGER"),
bigquery.SchemaField("q7", "INTEGER"),
bigquery.SchemaField("q8", "INTEGER"),
bigquery.SchemaField("q9", "INTEGER"),
bigquery.SchemaField("q10", "INTEGER"),
bigquery.SchemaField("q11", "INTEGER"),
bigquery.SchemaField("q12", "INTEGER"),
bigquery.SchemaField("q13", "INTEGER"),
bigquery.SchemaField("q14", "INTEGER"),
bigquery.SchemaField("q15", "INTEGER"),
bigquery.SchemaField("q16", "INTEGER"),
bigquery.SchemaField("q17", "INTEGER"),
bigquery.SchemaField("q18", "INTEGER"),
bigquery.SchemaField("q19", "INTEGER"),
bigquery.SchemaField("q20", "INTEGER"),
bigquery.SchemaField("q21", "INTEGER"),
bigquery.SchemaField("q22", "INTEGER"),
bigquery.SchemaField("q23", "INTEGER"),
bigquery.SchemaField("q24", "INTEGER"),
bigquery.S

In [222]:
muse_model_df['q1106']

learner
6051420109275136    None
5134423787831296    None
5798113272922112    None
5756901417549824    None
5630121163620352    None
5628773986729984    None
5637257553772544    None
5722679185244160    None
6125581443792896    None
5183635388891136    None
5688987666612224    None
5639491708518400    None
5729345880457216    None
5745528008605696    None
6463711870976000    None
5693802459168768    None
5682767362785280    None
5766696644116480    None
5633924927586304    None
5693526155198464    None
5138500986863616    None
5720843724259328    None
5666332938862592    None
5682738237538304    None
5731861556887552    None
5670200808570880    None
5708755706576896    None
5130576201777152    None
5728885882748928    None
5692767623708672    None
                    ... 
5145024048136192    None
5718042545750016    None
5659784879538176    None
5704698015252480    None
6261240749883392    None
6562469224906752    None
5191760145809408    None
5671751774109696    None
5650886118342656 

In [326]:
a = combined_tags_final.shape[0]
b = combined_true.shape[0]
c = combined_false.shape[0]
a + b + c

34662

In [327]:
combined_df

,learner,question,answer,last_modified,name,learner_name,tag,metatag,who,text,answer_type,answer_tag,metatags
37686,6298093637074944,40,True,2017-01-15 00:56:07.024890+00:00,nXJIar4lFgfgyXngcCoT434EdTwU2Vv4,Jack,[],[],parent,Do you have a graduate degree?,binary,None,None
2097,4999681536950272,152,True,2017-01-15 00:56:07.025080+00:00,cxKsk4j2ceAAj6WKpcsHDyXFgOmuKZs2,Rowan,[],[],child,Does {kid_name} cling to you or primary caregi...,scale,None,None
29958,5750743172644864,510,True,2017-01-15 00:56:07.025240+00:00,1hAPwIBJVEmwQLBVPXtxKy1qB6tTMwCA,Brady,[],[],child,Does {kid_name} anticipate likely outcomes of ...,scale,None,None
19756,5689015533568000,200,True,2017-01-15 00:56:07.025390+00:00,RjL7yAT8LV381Rr6nxlXiPkoZ8p76gtm,Alchemy,[yes],[answer],child,Does {kid_name} have temper tantrums?,scale,yes,answer
3077,5078095828615168,16,False,2017-01-15 00:56:07.025750+00:00,4ZZO7qdG1DOuSspZeaR3NVmyc7tGPlL6,Tank,[],[],parent,Do you have a hard time maintaining focus if a...,scale,None,None
34657,5947893009612800,686,True,2017-01-15 00:56:07.025930+00:00,fQtzzIhJIwaGPCoxPklyfEE8jNS6yR0T,Margo,[yes],[answer],parent,"In the last week, have you spent time playing ...",scale,yes,answer
1761,4920672459423744,891,False,2017-01-15 00:56:07.026480+00:00,01C3KSV4fkBSdjoelJSc3LgQUuVxGicK,Sam,[no],[answer],parent,Let’s assume a tax accountant and an artist ma...,scale,no,answer
17035,5673490367643648,773,True,2017-01-15 00:56:07.027430+00:00,dHDKkxi9v9TO7fK9y9XsLDd7pq9694JS,Oliver,[yes],[answer],child,"In the past week, has {kid_name} lost {their} ...",scale,yes,answer
38806,6496032036749312,821,True,2017-01-15 00:56:07.027690+00:00,AIezB9OSCjUnQ25cKtEACa8UBUskbfC7,Grayson,[],[],child,Do you and {kid_name} talk about things {kid_n...,scale,None,None
20345,5690996151025664,213,True,2017-01-15 00:56:07.027920+00:00,UofeP3xFcLVFp1zyZu19puM6o2BYXgbY,Lucas,[],[],child,Will {kid_name} scribble to draw?,scale,None,None
